In [1]:
import numpy as np
import pandas as pd
import requests
import csv
import ssl
from urllib.request import urlopen
from urllib.parse import quote_plus

In [2]:
pd.set_option('display.max_columns', None)

# Top_1

1. color를 원핫인코딩 변경
2. clothSexCommon이 1인 값은 남자와 여자 모두 1로 변경
3. user테이블에 그 user가 산 옷들 평균내서 fit ~ thickness ~ season까지 저장

top_1 = pd.read_csv('./musinsa_clothes_top_1.csv')

top_1.columns

top_1.info()

top_1.describe()

top_1 = top_1.drop_duplicates(subset = "clothId")
top_1 = top_1.reset_index(drop=True)

### smallCategory 원핫인코딩 변환

top_1.insert(4, "smallCategoryOthers", top_1['smallCategory'], True)
top_1.insert(4, "smallCategoryShort", top_1['smallCategory'], True)
top_1.insert(4, "smallCategoryLong", top_1['smallCategory'], True)
top_1.insert(4, "smallCategoryKnit", top_1['smallCategory'], True)
top_1.insert(4, "smallCategorySweatshirt", top_1['smallCategory'], True)
top_1.insert(4, "smallCategoryHoody", top_1['smallCategory'], True)
top_1.insert(4, "smallCategoryCollar", top_1['smallCategory'], True)
top_1.insert(4, "smallCategoryShirt", top_1['smallCategory'], True)
top_1.insert(4, "smallCategoryHalfshort", top_1['smallCategory'], True)

top_1['smallCategoryHalfshort'] = top_1['smallCategoryHalfshort'].apply(lambda x: 1 if x == 101 else 0)
top_1['smallCategoryShirt'] = top_1['smallCategoryShirt'].apply(lambda x: 1 if x == 102 else 0)
top_1['smallCategoryCollar'] = top_1['smallCategoryCollar'].apply(lambda x: 1 if x == 103 else 0)
top_1['smallCategoryHoody'] = top_1['smallCategoryHoody'].apply(lambda x: 1 if x == 104 else 0)
top_1['smallCategorySweatshirt'] = top_1['smallCategorySweatshirt'].apply(lambda x: 1 if x == 105 else 0)
top_1['smallCategoryKnit'] = top_1['smallCategoryKnit'].apply(lambda x: 1 if x == 106 else 0)
top_1['smallCategoryLong'] = top_1['smallCategoryLong'].apply(lambda x: 1 if x == 110 else 0)
top_1['smallCategoryShort'] = top_1['smallCategoryShort'].apply(lambda x: 1 if x == 111 else 0)
top_1['smallCategoryOthers'] = top_1['smallCategoryOthers'].apply(lambda x: 1 if x == 108 else 0)

top_1['largeCategory'].unique()

top_1['largeCategoryName'].unique()

top_1['smallCategory'].unique()

top_1['smallCategoryName'].unique()

top_1 = top_1.drop('smallCategory', axis=1)

top_1.columns

# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += top_1[top_1.columns[i]].value_counts()[1]
print(num)
print(top_1.shape)

### clothSexCommon 변환

top_1.head()

top_1['clothSexMen'] = top_1.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
top_1['clothSexWomen'] = top_1.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

top_1 = top_1.drop('clothSexCommon', axis=1)

top_1.columns

top_1['clothSexMen'] = top_1.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
top_1['clothSexWomen'] = top_1.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

print(top_1['clothSexMen'].value_counts())
print(top_1['clothSexWomen'].value_counts())
print(top_1.shape)

### color 원핫인코딩 변환

#### 스커트
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 원피스
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 바지
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 아우터
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

#### 상의
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

top_1.insert(14, "colorOthers", top_1['colorNo'], True)
top_1.insert(14, "colorPattern", top_1['colorNo'], True)
top_1.insert(14, "colorJean", top_1['colorNo'], True)
top_1.insert(14, "colorBeige", top_1['colorNo'], True)
top_1.insert(14, "colorBrown", top_1['colorNo'], True)
top_1.insert(14, "colorPurple", top_1['colorNo'], True)
top_1.insert(14, "colorBlue", top_1['colorNo'], True)
top_1.insert(14, "colorGreen", top_1['colorNo'], True)
top_1.insert(14, "colorYellow", top_1['colorNo'], True)
top_1.insert(14, "colorIvory", top_1['colorNo'], True)
top_1.insert(14, "colorOrange", top_1['colorNo'], True)
top_1.insert(14, "colorPink", top_1['colorNo'], True)
top_1.insert(14, "colorRed", top_1['colorNo'], True)
top_1.insert(14, "colorBlack", top_1['colorNo'], True)
top_1.insert(14, "colorGrey", top_1['colorNo'], True)
top_1.insert(14, "colorWhite", top_1['colorNo'], True)

top_1['colorWhite'] = top_1['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
top_1['colorGrey'] = top_1['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
top_1['colorBlack'] = top_1['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
top_1['colorRed'] = top_1['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
top_1['colorPink'] = top_1['colorPink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
top_1['colorOrange'] = top_1['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
top_1['colorIvory'] = top_1['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
top_1['colorYellow'] = top_1['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
top_1['colorGreen'] = top_1['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
top_1['colorBlue'] = top_1['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
top_1['colorPurple'] = top_1['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
top_1['colorBrown'] = top_1['colorBrown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
top_1['colorBeige'] = top_1['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
top_1['colorJean'] = top_1['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
top_1['colorPattern'] = top_1['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
top_1['colorOthers'] = top_1['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

top_1 = top_1.drop('colorNo', axis=1)

top_1.columns

top_1.insert(29, "colorName", True)

top_1['colorName'] = top_1.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
top_1['colorName'] = top_1.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

top_1['colorName'].value_counts()

### Price 전처리

top_1['clothPrice'] = top_1['clothPrice'].apply(lambda x: x.replace(",", ""))
top_1['clothPrice'] = top_1['clothPrice'].apply(lambda x: x.replace("원", ""))
top_1['clothPrice']

top_1['clothPrice'] = top_1['clothPrice'].astype('int')

top_1['clothReviewNo'] = top_1['clothReviewNo'].apply(lambda x: x.replace(",", ""))
top_1['clothReviewNo'] = top_1['clothReviewNo'].astype('int')
top_1['clothReviewNo']

top_1.index = top_1['clothId']
top_1 = top_1.drop('clothId', True)
print(top_1)

top_1.info()

# Top_2

top_2 = pd.read_csv('./musinsa_clothes_top_2.csv')

top_2.columns

top_2.info()

top_2.describe()

top_2 = top_2.drop_duplicates(subset = "clothId")
top_2 = top_2.reset_index(drop=True)

top_2['smallCategory'] = top_2['smallCategory'].astype('str')

top_2['largeCategory'].unique()

top_2['largeCategoryName'].unique()

top_2['smallCategory'].unique()

top_2['smallCategoryName'].unique()

top_2[top_2['smallCategoryName']=='\r\n            (나이키)        ']

top_2.loc[top_2.smallCategoryName=='\r\n            (나이키)        ', ('smallCategoryName')] = '반팔 티셔츠'

top_2[top_2['smallCategory']=="1s="]

top_2.loc[top_2.smallCategory=='1s=', ('smallCategory')] = '101'

top_2['smallCategory'] = top_2['smallCategory'].astype('int')

### smallCategory 원핫인코딩 변환

top_2.insert(4, "smallCategoryOthers", top_2['smallCategory'], True)
top_2.insert(4, "smallCategoryShort", top_2['smallCategory'], True)
top_2.insert(4, "smallCategoryLong", top_2['smallCategory'], True)
top_2.insert(4, "smallCategoryKnit", top_2['smallCategory'], True)
top_2.insert(4, "smallCategorySweatshirt", top_2['smallCategory'], True)
top_2.insert(4, "smallCategoryHoody", top_2['smallCategory'], True)
top_2.insert(4, "smallCategoryCollar", top_2['smallCategory'], True)
top_2.insert(4, "smallCategoryShirt", top_2['smallCategory'], True)
top_2.insert(4, "smallCategoryHalfshort", top_2['smallCategory'], True)

top_2['smallCategoryHalfshort'] = top_2['smallCategoryHalfshort'].apply(lambda x: 1 if x == 101 else 0)
top_2['smallCategoryShirt'] = top_2['smallCategoryShirt'].apply(lambda x: 1 if x == 102 else 0)
top_2['smallCategoryCollar'] = top_2['smallCategoryCollar'].apply(lambda x: 1 if x == 103 else 0)
top_2['smallCategoryHoody'] = top_2['smallCategoryHoody'].apply(lambda x: 1 if x == 104 else 0)
top_2['smallCategorySweatshirt'] = top_2['smallCategorySweatshirt'].apply(lambda x: 1 if x == 105 else 0)
top_2['smallCategoryKnit'] = top_2['smallCategoryKnit'].apply(lambda x: 1 if x == 106 else 0)
top_2['smallCategoryLong'] = top_2['smallCategoryLong'].apply(lambda x: 1 if x == 110 else 0)
top_2['smallCategoryShort'] = top_2['smallCategoryShort'].apply(lambda x: 1 if x == 111 else 0)
top_2['smallCategoryOthers'] = top_2['smallCategoryOthers'].apply(lambda x: 1 if x == 108 else 0)

top_2 = top_2.drop('smallCategory', axis=1)

top_2.columns

# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += top_2[top_2.columns[i]].value_counts()[1]
print(num)
print(top_2.shape)

### clothSexCommon 변환

top_2['clothSexMen'] = top_2.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
top_2['clothSexWomen'] = top_2.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

top_2 = top_2.drop('clothSexCommon', axis=1)

top_2.columns

top_2['clothSexMen'] = top_2.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
top_2['clothSexWomen'] = top_2.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

print(top_2['clothSexMen'].value_counts())
print(top_2['clothSexWomen'].value_counts())
print(top_2.shape)

### color 원핫인코딩 변환

#### 상의
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

top_2.insert(14, "colorOthers", top_2['colorNo'], True)
top_2.insert(14, "colorPattern", top_2['colorNo'], True)
top_2.insert(14, "colorJean", top_2['colorNo'], True)
top_2.insert(14, "colorBeige", top_2['colorNo'], True)
top_2.insert(14, "colorBrown", top_2['colorNo'], True)
top_2.insert(14, "colorPurple", top_2['colorNo'], True)
top_2.insert(14, "colorBlue", top_2['colorNo'], True)
top_2.insert(14, "colorGreen", top_2['colorNo'], True)
top_2.insert(14, "colorYellow", top_2['colorNo'], True)
top_2.insert(14, "colorIvory", top_2['colorNo'], True)
top_2.insert(14, "colorOrange", top_2['colorNo'], True)
top_2.insert(14, "colorPink", top_2['colorNo'], True)
top_2.insert(14, "colorRed", top_2['colorNo'], True)
top_2.insert(14, "colorBlack", top_2['colorNo'], True)
top_2.insert(14, "colorGrey", top_2['colorNo'], True)
top_2.insert(14, "colorWhite", top_2['colorNo'], True)

top_2['colorWhite'] = top_2['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
top_2['colorGrey'] = top_2['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
top_2['colorBlack'] = top_2['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
top_2['colorRed'] = top_2['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
top_2['colorPink'] = top_2['colorPink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
top_2['colorOrange'] = top_2['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
top_2['colorIvory'] = top_2['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
top_2['colorYellow'] = top_2['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
top_2['colorGreen'] = top_2['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
top_2['colorBlue'] = top_2['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
top_2['colorPurple'] = top_2['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
top_2['colorBrown'] = top_2['colorBrown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
top_2['colorBeige'] = top_2['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
top_2['colorJean'] = top_2['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
top_2['colorPattern'] = top_2['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
top_2['colorOthers'] = top_2['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

top_2 = top_2.drop('colorNo', axis=1)

print(top_2.columns)

# color도 모든 1의 값의 개수 == 행의 개수
for i in range(13, 29):
    print(top_2[top_2.columns[i]].value_counts())
print(top_2.shape)

top_2.insert(29, "colorName", True)

top_2['colorName'] = top_2.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
top_2['colorName'] = top_2.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

top_2['colorName'].value_counts()

### Price 전처리

top_2['clothPrice'] = top_2['clothPrice'].apply(lambda x: x.replace(",", ""))
top_2['clothPrice'] = top_2['clothPrice'].apply(lambda x: x.replace("원", ""))
top_2['clothPrice']

top_2['clothPrice'] = top_2['clothPrice'].astype('int')

top_2['clothReviewNo'] = top_2['clothReviewNo'].apply(lambda x: x.replace(",", ""))
top_2['clothReviewNo'] = top_2['clothReviewNo'].astype('int')
top_2['clothReviewNo']

top_2.index = top_2['clothId']
top_2 = top_2.drop('clothId', True)
print(top_2)

top_2.info()

# Top_3

top_3 = pd.read_csv('./musinsa_clothes_top_3.csv')

top_3.columns

top_3.info()

top_3.describe()

top_3 = top_3.drop_duplicates(subset = "clothId")
top_3 = top_3.reset_index(drop=True)

top_3['smallCategory'] = top_3['smallCategory'].astype('str')

top_3['largeCategory'].unique()

top_3['largeCategoryName'].unique()

top_3['smallCategory'].unique()

top_3['smallCategoryName'].unique()

top_3[top_3['smallCategoryName']=='\r\n            (나이키)        ']

top_3.loc[top_3.smallCategoryName=='\r\n            (로라로라)        ', ('smallCategoryName')] = '후드 티셔츠'

top_3[top_3['smallCategory']=="1s="]

top_3.loc[top_3.smallCategory=='1s=', ('smallCategory')] = '104'

top_3['smallCategory'] = top_3['smallCategory'].astype('int')

### smallCategory 원핫인코딩 변환

top_3.insert(4, "smallCategoryOthers", top_3['smallCategory'], True)
top_3.insert(4, "smallCategoryShort", top_3['smallCategory'], True)
top_3.insert(4, "smallCategoryLong", top_3['smallCategory'], True)
top_3.insert(4, "smallCategoryKnit", top_3['smallCategory'], True)
top_3.insert(4, "smallCategorySweatshirt", top_3['smallCategory'], True)
top_3.insert(4, "smallCategoryHoody", top_3['smallCategory'], True)
top_3.insert(4, "smallCategoryCollar", top_3['smallCategory'], True)
top_3.insert(4, "smallCategoryShirt", top_3['smallCategory'], True)
top_3.insert(4, "smallCategoryHalfshort", top_3['smallCategory'], True)

top_3['smallCategoryHalfshort'] = top_3['smallCategoryHalfshort'].apply(lambda x: 1 if x == 101 else 0)
top_3['smallCategoryShirt'] = top_3['smallCategoryShirt'].apply(lambda x: 1 if x == 102 else 0)
top_3['smallCategoryCollar'] = top_3['smallCategoryCollar'].apply(lambda x: 1 if x == 103 else 0)
top_3['smallCategoryHoody'] = top_3['smallCategoryHoody'].apply(lambda x: 1 if x == 104 else 0)
top_3['smallCategorySweatshirt'] = top_3['smallCategorySweatshirt'].apply(lambda x: 1 if x == 105 else 0)
top_3['smallCategoryKnit'] = top_3['smallCategoryKnit'].apply(lambda x: 1 if x == 106 else 0)
top_3['smallCategoryLong'] = top_3['smallCategoryLong'].apply(lambda x: 1 if x == 110 else 0)
top_3['smallCategoryShort'] = top_3['smallCategoryShort'].apply(lambda x: 1 if x == 111 else 0)
top_3['smallCategoryOthers'] = top_3['smallCategoryOthers'].apply(lambda x: 1 if x == 108 else 0)

top_3 = top_3.drop('smallCategory', axis=1)

top_3.columns

# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += top_3[top_3.columns[i]].value_counts()[1]
    print(top_3[top_3.columns[i]].value_counts())
print(num)
print(top_3.shape)

### clothSexCommon 변환

top_3['clothSexMen'] = top_3.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
top_3['clothSexWomen'] = top_3.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

top_3 = top_3.drop('clothSexCommon', axis=1)

top_3.columns

top_3['clothSexMen'] = top_3.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
top_3['clothSexWomen'] = top_3.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

print(top_3['clothSexMen'].value_counts())
print(top_3['clothSexWomen'].value_counts())
print(top_3.shape)

### color 원핫인코딩 변환

#### 상의
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

top_3.insert(14, "colorOthers", top_3['colorNo'], True)
top_3.insert(14, "colorPattern", top_3['colorNo'], True)
top_3.insert(14, "colorJean", top_3['colorNo'], True)
top_3.insert(14, "colorBeige", top_3['colorNo'], True)
top_3.insert(14, "colorBrown", top_3['colorNo'], True)
top_3.insert(14, "colorPurple", top_3['colorNo'], True)
top_3.insert(14, "colorBlue", top_3['colorNo'], True)
top_3.insert(14, "colorGreen", top_3['colorNo'], True)
top_3.insert(14, "colorYellow", top_3['colorNo'], True)
top_3.insert(14, "colorIvory", top_3['colorNo'], True)
top_3.insert(14, "colorOrange", top_3['colorNo'], True)
top_3.insert(14, "colorPink", top_3['colorNo'], True)
top_3.insert(14, "colorRed", top_3['colorNo'], True)
top_3.insert(14, "colorBlack", top_3['colorNo'], True)
top_3.insert(14, "colorGrey", top_3['colorNo'], True)
top_3.insert(14, "colorWhite", top_3['colorNo'], True)

top_3['colorWhite'] = top_3['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
top_3['colorGrey'] = top_3['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
top_3['colorBlack'] = top_3['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
top_3['colorRed'] = top_3['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
top_3['colorPink'] = top_3['colorPink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
top_3['colorOrange'] = top_3['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
top_3['colorIvory'] = top_3['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
top_3['colorYellow'] = top_3['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
top_3['colorGreen'] = top_3['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
top_3['colorBlue'] = top_3['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
top_3['colorPurple'] = top_3['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
top_3['colorBrown'] = top_3['colorBrown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
top_3['colorBeige'] = top_3['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
top_3['colorJean'] = top_3['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
top_3['colorPattern'] = top_3['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
top_3['colorOthers'] = top_3['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

print(top_3.columns)

top_3 = top_3.drop('colorNo', axis=1)

# color도 모든 1의 값의 개수 == 행의 개수
for i in range(13, 29):
    print(top_3[top_3.columns[i]].value_counts())
print(top_3.shape)

top_3.insert(29, "colorName", True)

top_3['colorName'] = top_3.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
top_3['colorName'] = top_3.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

top_3['colorName'].value_counts()

### Price 전처리

top_3['clothPrice'] = top_3['clothPrice'].apply(lambda x: x.replace(",", ""))
top_3['clothPrice'] = top_3['clothPrice'].apply(lambda x: x.replace("원", ""))
top_3['clothPrice']

top_3['clothPrice'] = top_3['clothPrice'].astype('int')

top_3['clothReviewNo'] = top_3['clothReviewNo'].apply(lambda x: x.replace(",", ""))
top_3['clothReviewNo'] = top_3['clothReviewNo'].astype('int')
top_3['clothReviewNo']

top_3.index = top_3['clothId']
top_3 = top_3.drop('clothId', True)
print(top_3)

top_3.info()

# Top_4

top_4 = pd.read_csv('./musinsa_clothes_top_4.csv')

top_4.columns

top_4.info()

top_4.describe()

top_4 = top_4.drop_duplicates(subset = "clothId")
top_4 = top_4.reset_index(drop=True)

top_4['smallCategory'] = top_4['smallCategory'].astype('str')

top_4['largeCategory'].unique()

top_4['largeCategoryName'].unique()

top_4['smallCategory'].unique()

top_4['smallCategoryName'].unique()

top_4[top_4['smallCategoryName']=='\n            (로라로라)        ']

top_4.loc[top_4.smallCategoryName=='\n            (로라로라)        ', ('smallCategoryName')] = '후드 티셔츠'

top_4[top_4['smallCategory']=="1s="]

top_4.loc[top_4.smallCategory=='1s=', ('smallCategory')] = '104'

top_4['smallCategory'] = top_4['smallCategory'].astype('int')

### smallCategory 원핫인코딩 변환

top_4.insert(4, "smallCategoryOthers", top_4['smallCategory'], True)
top_4.insert(4, "smallCategoryShort", top_4['smallCategory'], True)
top_4.insert(4, "smallCategoryLong", top_4['smallCategory'], True)
top_4.insert(4, "smallCategoryKnit", top_4['smallCategory'], True)
top_4.insert(4, "smallCategorySweatshirt", top_4['smallCategory'], True)
top_4.insert(4, "smallCategoryHoody", top_4['smallCategory'], True)
top_4.insert(4, "smallCategoryCollar", top_4['smallCategory'], True)
top_4.insert(4, "smallCategoryShirt", top_4['smallCategory'], True)
top_4.insert(4, "smallCategoryHalfshort", top_4['smallCategory'], True)

top_4['smallCategoryHalfshort'] = top_4['smallCategoryHalfshort'].apply(lambda x: 1 if x == 101 else 0)
top_4['smallCategoryShirt'] = top_4['smallCategoryShirt'].apply(lambda x: 1 if x == 102 else 0)
top_4['smallCategoryCollar'] = top_4['smallCategoryCollar'].apply(lambda x: 1 if x == 103 else 0)
top_4['smallCategoryHoody'] = top_4['smallCategoryHoody'].apply(lambda x: 1 if x == 104 else 0)
top_4['smallCategorySweatshirt'] = top_4['smallCategorySweatshirt'].apply(lambda x: 1 if x == 105 else 0)
top_4['smallCategoryKnit'] = top_4['smallCategoryKnit'].apply(lambda x: 1 if x == 106 else 0)
top_4['smallCategoryLong'] = top_4['smallCategoryLong'].apply(lambda x: 1 if x == 110 else 0)
top_4['smallCategoryShort'] = top_4['smallCategoryShort'].apply(lambda x: 1 if x == 111 else 0)
top_4['smallCategoryOthers'] = top_4['smallCategoryOthers'].apply(lambda x: 1 if x == 108 else 0)

top_4 = top_4.drop('smallCategory', axis=1)

top_4.columns

# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += top_4[top_4.columns[i]].value_counts()[1]
    print(top_4[top_4.columns[i]].value_counts())
print(num)
print(top_4.shape)

### clothSexCommon 변환

top_4['clothSexMen'] = top_4.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
top_4['clothSexWomen'] = top_4.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

top_4 = top_4.drop('clothSexCommon', axis=1)

top_4.columns

top_4['clothSexMen'] = top_4.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
top_4['clothSexWomen'] = top_4.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

print(top_4['clothSexMen'].value_counts())
print(top_4['clothSexWomen'].value_counts())
print(top_4.shape)

### color 원핫인코딩 변환

#### 상의
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

top_4.insert(14, "colorOthers", top_4['colorNo'], True)
top_4.insert(14, "colorPattern", top_4['colorNo'], True)
top_4.insert(14, "colorJean", top_4['colorNo'], True)
top_4.insert(14, "colorBeige", top_4['colorNo'], True)
top_4.insert(14, "colorBrown", top_4['colorNo'], True)
top_4.insert(14, "colorPurple", top_4['colorNo'], True)
top_4.insert(14, "colorBlue", top_4['colorNo'], True)
top_4.insert(14, "colorGreen", top_4['colorNo'], True)
top_4.insert(14, "colorYellow", top_4['colorNo'], True)
top_4.insert(14, "colorIvory", top_4['colorNo'], True)
top_4.insert(14, "colorOrange", top_4['colorNo'], True)
top_4.insert(14, "colorPink", top_4['colorNo'], True)
top_4.insert(14, "colorRed", top_4['colorNo'], True)
top_4.insert(14, "colorBlack", top_4['colorNo'], True)
top_4.insert(14, "colorGrey", top_4['colorNo'], True)
top_4.insert(14, "colorWhite", top_4['colorNo'], True)

top_4['colorWhite'] = top_4['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
top_4['colorGrey'] = top_4['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
top_4['colorBlack'] = top_4['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
top_4['colorRed'] = top_4['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
top_4['colorPink'] = top_4['colorPink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
top_4['colorOrange'] = top_4['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
top_4['colorIvory'] = top_4['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
top_4['colorYellow'] = top_4['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
top_4['colorGreen'] = top_4['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
top_4['colorBlue'] = top_4['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
top_4['colorPurple'] = top_4['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
top_4['colorBrown'] = top_4['colorBrown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
top_4['colorBeige'] = top_4['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
top_4['colorJean'] = top_4['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
top_4['colorPattern'] = top_4['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
top_4['colorOthers'] = top_4['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

print(top_4.columns)

top_4 = top_4.drop('colorNo', axis=1)

# color도 모든 1의 값의 개수 == 행의 개수
for i in range(13, 29):
    print(top_4[top_4.columns[i]].value_counts())
print(top_4.shape)

top_4.insert(29, "colorName", True)

top_4['colorName'] = top_4.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
top_4['colorName'] = top_4.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

top_4['colorName'].value_counts()

### Price 전처리

top_4['clothPrice'] = top_4['clothPrice'].apply(lambda x: x.replace(",", ""))
top_4['clothPrice'] = top_4['clothPrice'].apply(lambda x: x.replace("원", ""))
top_4['clothPrice']

top_4['clothPrice'] = top_4['clothPrice'].astype('int')

top_4['clothReviewNo'] = top_4['clothReviewNo'].apply(lambda x: x.replace(",", ""))
top_4['clothReviewNo'] = top_4['clothReviewNo'].astype('int')
top_4['clothReviewNo']

top_4.index = top_4['clothId']
top_4 = top_4.drop('clothId', True)
top_4

top_4.info()

df_top = pd.concat([top_1, top_2, top_3, top_4], axis = 0)

df_top.shape

df_top.head()

df_top.to_csv('final_top.csv',
                 sep=',',
                 float_format = '%.1f',
                 encoding = 'utf-8-sig')

# TOP

In [3]:
top_1 = pd.read_csv('./musinsa_clothes_top_1.csv')
top_2 = pd.read_csv('./musinsa_clothes_top_2.csv')
top_3 = pd.read_csv('./musinsa_clothes_top_3.csv')
top_4 = pd.read_csv('./musinsa_clothes_top_4.csv')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_17388\3553540150.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  top_2 = pd.read_csv('./musinsa_clothes_top_2.csv')
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_17388\3553540150.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  top_3 = pd.read_csv('./musinsa_clothes_top_3.csv')
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_17388\3553540150.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  top_4 = pd.read_csv('./musinsa_clothes_top_4.csv')


In [4]:
top = pd.concat([top_1, top_2, top_3, top_4], axis=0)

In [5]:
top.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [6]:
top.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158232 entries, 0 to 34325
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   clothId            158232 non-null  int64  
 1   largeCategory      158232 non-null  int64  
 2   largeCategoryName  158232 non-null  object 
 3   smallCategory      158232 non-null  object 
 4   smallCategoryName  158232 non-null  object 
 5   colorNo            158232 non-null  int64  
 6   clothName          158232 non-null  object 
 7   brand              158232 non-null  object 
 8   image              158232 non-null  object 
 9   clothPrice         158232 non-null  object 
 10  date               158232 non-null  int64  
 11  hashtags           158232 non-null  object 
 12  clothSexMen        158232 non-null  int64  
 13  clothSexWomen      158232 non-null  int64  
 14  clothSexCommon     158232 non-null  int64  
 15  clothRate          158232 non-null  float64
 16  clo

In [7]:
top.describe()

,clothId,largeCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,1.582320e+05,158232.0,158232.000000,1.582320e+05,158232.000000,158232.000000,158232.000000,158232.000000,158232.000000,158232.000000,158232.000000,158232.000000,158232.000000,158232.000000,158232.000000,158232.000000,158232.000000
mean,1.783307e+06,1.0,12.773921,2.020522e+07,0.675533,0.285663,0.038526,2.319507,1.093837,0.616904,1.018391,1.272126,0.898282,0.204636,0.134240,0.206286,0.147492
std,5.163266e+05,0.0,14.117192,1.361415e+04,0.468177,0.451731,0.192462,2.374161,1.845494,1.104713,1.712370,2.058405,1.492389,0.403437,0.340911,0.404639,0.354597
min,1.039000e+04,1.0,1.000000,2.011061e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.482432e+06,1.0,2.000000,2.020061e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.923626e+06,1.0,5.000000,2.021043e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.180262e+06,1.0,23.000000,2.021102e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,3.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000
max,2.413604e+06,1.0,71.000000,2.022031e+07,1.000000,1.000000,1.000000,6.700000,5.000000,6.000000,6.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
top = top.drop_duplicates(subset = "clothId")
top = top.reset_index(drop=True)

In [9]:
top['smallCategory'] = top['smallCategory'].astype('str')

In [10]:
top['largeCategory'].unique()

array([1], dtype=int64)

In [11]:
top['largeCategoryName'].unique()

array(['상의'], dtype=object)

In [12]:
top['smallCategory'].unique()

array(['101', '103', '111', '110', '108', '105', '106', '102', '104',
       '1s='], dtype=object)

In [13]:
top['smallCategoryName'].unique()

array(['반팔 티셔츠', '피케/카라 티셔츠', '민소매 티셔츠', '긴팔 티셔츠', '기타 상의', '맨투맨/스웨트셔츠',
       '니트/스웨터', '셔츠/블라우스', '후드 티셔츠', '\r\n            (나이키)        ',
       '\r\n            (로라로라)        ', '\n            (로라로라)        '],
      dtype=object)

In [14]:
top[top['smallCategory']=="1s="]

,clothId,largeCategory,largeCategoryName,smallCategory,smallCategoryName,colorNo,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothSexCommon,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
75195,1385204,1,상의,1s=,\r\n (나이키),2,반팔티 레전드 2.0 NI-TT7 택1 718833,나이키,//image.msscdn.net/images/goods_img/20200403/1...,"38,900원",20200403,[],1,0,0,5.0,1,0,0,0,0,0,0,0,0,0
102957,1541525,1,상의,1s=,\r\n (로라로라),32,(HD-20521) ROLA SIGNATURE HOOD T-SHIRT MINT,로라로라,//image.msscdn.net/images/goods_img/20200807/1...,"63,200원",20200807,"['시그니처후드', '로고후드', '파스텔후드', '자수후드', '후드티', '로고...",0,1,0,4.7,108,0,0,0,0,0,0,0,0,0
136650,1541526,1,상의,1s=,\n (로라로라),39,(HD-20521) ROLA SIGNATURE HOOD T-SHIRT LAVENDER,로라로라,//image.msscdn.net/images/goods_img/20200807/1...,"63,200원",20200807,"['시그니처후드', '로고후드', '파스텔후드', '자수후드', '후드티', '로고...",0,1,0,4.7,108,0,0,0,0,0,0,0,0,0


In [15]:
top.loc[top.clothId==1385204, 'smallCategory'] = '101'
top.loc[top.clothId==1385204, 'smallCategoryName'] = '반팔 티셔츠'
top.loc[top.clothId==1541525, 'smallCategory'] = '104'
top.loc[top.clothId==1541525, 'smallCategoryName'] = '후드 티셔츠'
top.loc[top.clothId==1541526, 'smallCategory'] = '104'
top.loc[top.clothId==1541526, 'smallCategoryName'] = '후드 티셔츠'

In [16]:
top['smallCategory'] = top['smallCategory'].astype('int')

In [17]:
top['smallCategory'].value_counts()

101    49337
105    31165
102    19847
104    18336
110    15180
106    13918
103     5299
111     2981
108     1577
Name: smallCategory, dtype: int64

In [18]:
print(49337+31165+19847+18336+15180+13918+5299+2981+1577)
print(top.shape)

157640
(157640, 26)


### smallCategory 원핫인코딩 변환

In [19]:
top.insert(4, "smallCategoryOthers", top['smallCategory'], True)
top.insert(4, "smallCategoryShort", top['smallCategory'], True)
top.insert(4, "smallCategoryLong", top['smallCategory'], True)
top.insert(4, "smallCategoryKnit", top['smallCategory'], True)
top.insert(4, "smallCategorySweatshirt", top['smallCategory'], True)
top.insert(4, "smallCategoryHoody", top['smallCategory'], True)
top.insert(4, "smallCategoryCollar", top['smallCategory'], True)
top.insert(4, "smallCategoryShirt", top['smallCategory'], True)
top.insert(4, "smallCategoryHalfshort", top['smallCategory'], True)

In [20]:
top['smallCategoryHalfshort'] = top['smallCategoryHalfshort'].apply(lambda x: 1 if x == 101 else 0)
top['smallCategoryShirt'] = top['smallCategoryShirt'].apply(lambda x: 1 if x == 102 else 0)
top['smallCategoryCollar'] = top['smallCategoryCollar'].apply(lambda x: 1 if x == 103 else 0)
top['smallCategoryHoody'] = top['smallCategoryHoody'].apply(lambda x: 1 if x == 104 else 0)
top['smallCategorySweatshirt'] = top['smallCategorySweatshirt'].apply(lambda x: 1 if x == 105 else 0)
top['smallCategoryKnit'] = top['smallCategoryKnit'].apply(lambda x: 1 if x == 106 else 0)
top['smallCategoryLong'] = top['smallCategoryLong'].apply(lambda x: 1 if x == 110 else 0)
top['smallCategoryShort'] = top['smallCategoryShort'].apply(lambda x: 1 if x == 111 else 0)
top['smallCategoryOthers'] = top['smallCategoryOthers'].apply(lambda x: 1 if x == 108 else 0)

In [21]:
top = top.drop('smallCategory', axis=1)

In [22]:
top.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [23]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 12):
    num += top[top.columns[i]].value_counts()[1]
    print(top[top.columns[i]].value_counts())
print(num)
print(top.shape)

0    108303
1     49337
Name: smallCategoryHalfshort, dtype: int64
0    137793
1     19847
Name: smallCategoryShirt, dtype: int64
0    152341
1      5299
Name: smallCategoryCollar, dtype: int64
0    139304
1     18336
Name: smallCategoryHoody, dtype: int64
0    126475
1     31165
Name: smallCategorySweatshirt, dtype: int64
0    143722
1     13918
Name: smallCategoryKnit, dtype: int64
0    142460
1     15180
Name: smallCategoryLong, dtype: int64
0    154659
1      2981
Name: smallCategoryShort, dtype: int64
0    156063
1      1577
Name: smallCategoryOthers, dtype: int64
157640
(157640, 34)


### clothSexCommon 변환

In [24]:
top['clothSexMen'] = top.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
top['clothSexWomen'] = top.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

In [25]:
top = top.drop('clothSexCommon', axis=1)

In [26]:
top.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [27]:
top['clothSexMen'] = top.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
top['clothSexWomen'] = top.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

In [28]:
print(top['clothSexMen'].value_counts())
print(top['clothSexWomen'].value_counts())
print(top.shape)

1    112546
0     45094
Name: clothSexMen, dtype: int64
0    106459
1     51181
Name: clothSexWomen, dtype: int64
(157640, 33)


### color 원핫인코딩 변환

#### 아우터
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [29]:
top.insert(14, "colorOthers", top['colorNo'], True)
top.insert(14, "colorPattern", top['colorNo'], True)
top.insert(14, "colorJean", top['colorNo'], True)
top.insert(14, "colorBeige", top['colorNo'], True)
top.insert(14, "colorBrown", top['colorNo'], True)
top.insert(14, "colorPurple", top['colorNo'], True)
top.insert(14, "colorBlue", top['colorNo'], True)
top.insert(14, "colorGreen", top['colorNo'], True)
top.insert(14, "colorYellow", top['colorNo'], True)
top.insert(14, "colorIvory", top['colorNo'], True)
top.insert(14, "colorOrange", top['colorNo'], True)
top.insert(14, "colorPink", top['colorNo'], True)
top.insert(14, "colorRed", top['colorNo'], True)
top.insert(14, "colorBlack", top['colorNo'], True)
top.insert(14, "colorGrey", top['colorNo'], True)
top.insert(14, "colorWhite", top['colorNo'], True)

In [30]:
top['colorWhite'] = top['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
top['colorGrey'] = top['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
top['colorBlack'] = top['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
top['colorRed'] = top['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
top['colorPink'] = top['colorPink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
top['colorOrange'] = top['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
top['colorIvory'] = top['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
top['colorYellow'] = top['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
top['colorGreen'] = top['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
top['colorBlue'] = top['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
top['colorPurple'] = top['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
top['colorBrown'] = top['colorBrown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
top['colorBeige'] = top['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
top['colorJean'] = top['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
top['colorPattern'] = top['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
top['colorOthers'] = top['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [31]:
top = top.drop('colorNo', axis=1)

In [32]:
print(top.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'clothName', 'brand',
       'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')


In [33]:
# color도 모든 1의 값의 개수 == 행의 개수
num = 0
for i in range(13, 29):
    num+=top[top.columns[i]].value_counts()[1]
    print(top[top.columns[i]].value_counts())
print(num)
print(top.shape)

0    134982
1     22658
Name: colorWhite, dtype: int64
0    137043
1     20597
Name: colorGrey, dtype: int64
0    121855
1     35785
Name: colorBlack, dtype: int64
0    154478
1      3162
Name: colorRed, dtype: int64
0    153415
1      4225
Name: colorPink, dtype: int64
0    155132
1      2508
Name: colorOrange, dtype: int64
0    149883
1      7757
Name: colorIvory, dtype: int64
0    153687
1      3953
Name: colorYellow, dtype: int64
0    145792
1     11848
Name: colorGreen, dtype: int64
0    135975
1     21665
Name: colorBlue, dtype: int64
0    154304
1      3336
Name: colorPurple, dtype: int64
0    154699
1      2941
Name: colorBrown, dtype: int64
0    149941
1      7699
Name: colorBeige, dtype: int64
0    157113
1       527
Name: colorJean, dtype: int64
0    154801
1      2839
Name: colorPattern, dtype: int64
0    151500
1      6140
Name: colorOthers, dtype: int64
157640
(157640, 48)


In [34]:
top.insert(29, "colorName", True)

In [35]:
top.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [36]:
top['colorName'] = top.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
top['colorName'] = top.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
top['colorName'] = top.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
top['colorName'] = top.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
top['colorName'] = top.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
top['colorName'] = top.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
top['colorName'] = top.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
top['colorName'] = top.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
top['colorName'] = top.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
top['colorName'] = top.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
top['colorName'] = top.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
top['colorName'] = top.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
top['colorName'] = top.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
top['colorName'] = top.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
top['colorName'] = top.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
top['colorName'] = top.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

In [37]:
top['colorName'].value_counts()

black      35785
white      22658
blue       21665
grey       20597
green      11848
ivory       7757
beige       7699
others      6140
pink        4225
yellow      3953
purple      3336
red         3162
brown       2941
pattern     2839
orange      2508
jean         527
Name: colorName, dtype: int64

### Price 전처리

In [38]:
top['clothPrice'] = top['clothPrice'].apply(lambda x: x.replace(",", ""))
top['clothPrice'] = top['clothPrice'].apply(lambda x: x.replace("원", ""))
top['clothPrice']

0         59000
1         43000
2         43000
3         26600
4         24000
          ...  
157635    58000
157636    66400
157637     8900
157638    69300
157639    29900
Name: clothPrice, Length: 157640, dtype: object

In [39]:
top['clothPrice'] = top['clothPrice'].astype('int')

In [40]:
top['clothReviewNo'] = top['clothReviewNo'].apply(lambda x: x.replace(",", ""))
top['clothReviewNo'] = top['clothReviewNo'].astype('int')
top['clothReviewNo']

0         0
1         0
2         2
3         1
4         0
         ..
157635    0
157636    0
157637    0
157638    0
157639    0
Name: clothReviewNo, Length: 157640, dtype: int32

### Index를 NewGoodsNo로 변경

In [41]:
sub_top = pd.read_json('./size_top.json')
sub_top

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,10390,M,1039000230446,230446,"[{'text': '문구', 'value': 1}, {'text': '맘', 'va...",160.000000,65.000000,0.0000,1.0000,0,0,0,0,0,0,0,0,0,1,0,0
1,10396,M,1039600198495,198495,"[{'text': '구매', 'value': 2}, {'text': '프린팅', '...",164.000000,55.000000,0.5000,0.5000,0,0,0,0,0,0,0,0,1,1,0,0
2,18716,XXL,1871600198467,198467,"[{'text': '잠옷', 'value': 4}, {'text': '집', 'va...",175.000000,88.000000,1.0000,0.0000,0,0,2,0,0,0,0,0,0,0,0,0
3,18724,S,1872400194701,194701,"[{'text': '외', 'value': 1}, {'text': '제일', 'va...",162.000000,48.000000,0.0000,1.0000,0,0,0,0,1,0,0,0,0,0,0,0
4,23466,S,2346600231696,231696,"[{'text': '생각', 'value': 1}, {'text': '치', 'va...",160.000000,50.000000,1.0000,0.0000,0,0,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150225,2408672,M,2408672154785,154785,"[{'text': '핏', 'value': 2}, {'text': '디테일', 'v...",176.375000,70.437500,0.9375,0.0625,0,1,0,1,2,4,1,1,0,4,0,2
150226,2408675,M,2408675213992,213992,"[{'text': '느낌', 'value': 6}, {'text': '스타일', '...",171.315789,65.105263,1.0000,0.0000,0,0,1,3,2,2,4,0,1,4,2,0
150227,2409261,WOMEN-ONE,2409261194715,194715,"[{'text': '유니크', 'value': 1}, {'text': '바로', '...",168.400000,60.400000,0.8000,0.2000,0,0,0,0,0,1,0,0,0,3,1,0
150228,2410927,beige/L,2410927248936,248936,"[{'text': '디자인', 'value': 2}, {'text': '앞', 'v...",179.625000,77.625000,1.0000,0.0000,0,2,1,2,0,2,2,1,3,2,1,0


In [42]:
top = pd.merge(top, sub_top, how="left", left_on="clothId", right_on="goodsNo")
top.shape

(249606, 70)

In [43]:
top.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsNo', 'goodsSize', 'newGoodsNo',
       'index', 'reviewNoun', 'userHeight', 'userWeight', 'userSexMen',
       'userSexWomen', 'month1', 'month2', 'mon

In [44]:
top = top.drop(["goodsNo", "index"], axis=1)
top.shape

(249606, 68)

In [45]:
# 249606 : 반복문을 통해 변화된 skirt 행 개수
# 157640 : 원래의 skirt 행 개수
# 91966 : 새로 생긴 skirt 행 개수
249606-157640

91966

In [46]:
150230-58264

91966

In [47]:
len(sub_top)

150230

In [48]:
len(sub_top.goodsNo.unique())

58264

In [49]:
from copy import deepcopy
b = deepcopy(top)

In [50]:
b.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'newGoodsNo', 'reviewNoun',
       'userHeight', 'userWeight', 'userSexMen', 'userSexWomen', 'month1',
       'month2', 'month3', 'month4', 'mon

In [51]:
col_list_1 = ['newGoodsNo', 'userSexMen', 'userSexWomen']
col_list_2 = ['goodsSize', 'reviewNoun', 'userHeight', 'userWeight', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12']
check_list_1 = ['clothId',  'clothSexMen', 'clothSexWomen']
check_list_2 = ['Free', [], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(len(col_list_1)):
    b[col_list_1[i]] = b[col_list_1[i]].fillna(0)
    b[col_list_1[i]] = b.apply(lambda x: x[check_list_1[i]] if x[col_list_1[i]]==0 else x[col_list_1[i]], axis=1)
    if i==0:
        b[col_list_1[i]] = b[col_list_1[i]].astype('int64')
    
for i in range(len(col_list_2)):
    b[col_list_2[i]] = b[col_list_2[i]].fillna(0)
    b[col_list_2[i]] = b.apply(lambda x: check_list_2[i] if x[col_list_2[i]]==0 else x[col_list_2[i]], axis=1)
    if i == 0:
        b[col_list_2[i]] = b[col_list_2[i]].astype('str')
    elif i >= 3:
        b[col_list_2[i]] = b[col_list_2[i]].astype('int64')

In [52]:
top = deepcopy(b)

In [53]:
# 이미지 null 값 확인
print(len(top[top.image.isnull()]))

0


In [54]:
# hashtags 전처리
top.hashtags = top.apply(lambda x: x.hashtags.replace("['", "").replace("']", "").replace("'", "").replace(" ", "").split(","), axis=1)

In [55]:
top.userHeight = top.userHeight.astype('float')
top.userWeight = top.userWeight.astype('float')
top.userSexMen = top.userSexMen.astype('float')
top.userSexWomen = top.userSexWomen.astype('float')

In [56]:
top.reviewNoun.iloc[2]

[{'text': '하의', 'value': 2},
 {'text': '길이', 'value': 1},
 {'text': '생각', 'value': 1},
 {'text': '키', 'value': 1},
 {'text': '덩', 'value': 1},
 {'text': '치가', 'value': 1},
 {'text': '편이', 'value': 1},
 {'text': '정말', 'value': 1},
 {'text': '반바지', 'value': 1},
 {'text': '실종', 'value': 1},
 {'text': '패션', 'value': 1},
 {'text': '디자인', 'value': 1}]

In [57]:
top.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'newGoodsNo', 'reviewNoun',
       'userHeight', 'userWeight', 'userSexMen', 'userSexWomen', 'month1',
       'month2', 'month3', 'month4', 'mon

In [58]:
top.columns = ['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'clothImg', 'clothPrice', 'date', 'clothHashtags', 'clothMale',
       'clothFemale', 'clothRate', 'clothReviewCnt', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'newClothId', 'reviewNoun', 'userHeight',
       'userWeight', 'userMale', 'userFemale', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12']

In [59]:
top = top[['newClothId', 'clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHalfshort', 'smallCategoryShirt', 'smallCategoryCollar',
       'smallCategoryHoody', 'smallCategorySweatshirt', 'smallCategoryKnit',
       'smallCategoryLong', 'smallCategoryShort', 'smallCategoryOthers',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'clothImg', 'clothPrice', 'date', 'clothHashtags', 'clothMale',
       'clothFemale', 'clothRate', 'clothReviewCnt', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'reviewNoun', 'userHeight',
       'userWeight', 'userMale', 'userFemale', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12']]

In [60]:
top.shape

(249606, 68)

In [61]:
top.to_json('final_top.json', orient='records', force_ascii=False)

In [62]:
top.to_csv('final_top.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

# Outer

In [63]:
outer = pd.read_csv('./musinsa_clothes_outer.csv')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_17388\3755538683.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  outer = pd.read_csv('./musinsa_clothes_outer.csv')


In [64]:
outer.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [65]:
outer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55039 entries, 0 to 55038
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            55039 non-null  int64  
 1   largeCategory      55039 non-null  int64  
 2   largeCategoryName  55039 non-null  object 
 3   smallCategory      55039 non-null  object 
 4   smallCategoryName  55039 non-null  object 
 5   colorNo            55039 non-null  int64  
 6   clothName          55039 non-null  object 
 7   brand              55039 non-null  object 
 8   image              55039 non-null  object 
 9   clothPrice         55039 non-null  object 
 10  date               55039 non-null  int64  
 11  hashtags           55039 non-null  object 
 12  clothSexMen        55039 non-null  int64  
 13  clothSexWomen      55039 non-null  int64  
 14  clothSexCommon     55039 non-null  int64  
 15  clothRate          55039 non-null  float64
 16  clothReviewNo      550

In [66]:
outer.describe()

,clothId,largeCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,5.503900e+04,55039.0,55039.000000,5.503900e+04,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.000000,55039.00000
mean,1.935946e+06,2.0,13.667290,2.020825e+07,0.585512,0.372118,0.042261,2.237203,1.057632,0.648177,0.734316,1.297444,0.965915,0.192409,0.045095,0.217864,0.17315
std,4.423802e+05,0.0,14.269903,1.106843e+04,0.492638,0.483374,0.201186,2.377868,1.794288,1.184583,1.389274,2.131908,1.641572,0.394196,0.207515,0.412798,0.37838
min,1.878100e+04,2.0,1.000000,2.012051e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,1.682430e+06,2.0,2.000000,2.020111e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,2.117505e+06,2.0,5.000000,2.021091e+07,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,2.259102e+06,2.0,24.000000,2.021121e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,1.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.00000
max,2.413481e+06,2.0,71.000000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,5.000000,6.000000,6.000000,1.000000,1.000000,1.000000,1.00000


In [67]:
outer = outer.drop_duplicates(subset = "clothId")
outer = outer.reset_index(drop=True)

In [68]:
outer['smallCategory'] = outer['smallCategory'].astype('str')

In [69]:
outer['largeCategory'].unique()

array([2], dtype=int64)

In [70]:
outer['largeCategoryName'].unique()

array(['아우터'], dtype=object)

In [71]:
outer['smallCategory'].unique()

array(['218', '220', '201', '212', '206', '221', '213', '207', '222',
       '215', '219', '223', '209', '217', '216', '203', '214', '225',
       '202', '224', '208', '204', '2s='], dtype=object)

In [72]:
outer['smallCategoryName'].unique()

array(['트레이닝 재킷', '카디건', '블루종/MA-1', '숏패딩/숏헤비 아우터', '나일론/코치  재킷', '베스트',
       '롱패딩/롱헤비 아우터', '겨울 싱글 코트', '후드 집업', '기타 아우터', '아노락 재킷', '플리스/뽀글이',
       '겨울 기타 코트', '트러커 재킷', '패딩 베스트', '슈트/블레이저 재킷', '사파리/헌팅 재킷', '무스탕/퍼',
       '레더/라이더스 재킷', '겨울 더블 코트', '환절기 코트', '스타디움 재킷',
       '\r\n            (휠라)        '], dtype=object)

In [73]:
outer[outer['smallCategoryName']=='\r\n            (휠라)        ']

,clothId,largeCategory,largeCategoryName,smallCategory,smallCategoryName,colorNo,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothSexCommon,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
11366,1532828,2,아우터,2s=,\r\n (휠라),23,에센셜 리니어 보아 후드 자켓 (FE2JKD6101XCRM),휠라,//image.msscdn.net/images/goods_img/20200730/1...,"79,000원",20200730,"['보아', '자켓', '플리스', '후드', '후리스', '캐리오버', '온라인전...",0,0,1,4.8,355,0,0,0,0,0,0,0,0,0
16581,1532833,2,아우터,2s=,\r\n (휠라),3,리니어 와펜 본딩 보아 자켓 (FE2JKC6102XGRY),휠라,//image.msscdn.net/images/goods_img/20200730/1...,"69,000원",20200730,"['보아', '자켓', '집업', '플리스', '후리스', '온라인전용']",0,0,1,4.6,483,0,0,0,0,0,0,0,0,0
29633,1532832,2,아우터,2s=,\r\n (휠라),2,우븐 포켓 보아 자켓 (FE2JKD6103XBLK),휠라,//image.msscdn.net/images/goods_img/20200730/1...,"89,000원",20200730,"['보아', '자켓', '집업', '플리스', '후리스', '캐리오버', '온라인전용']",0,1,0,4.6,483,0,0,0,0,0,0,0,0,0


In [74]:
outer.loc[outer.smallCategoryName=='\r\n            (휠라)        ', ('smallCategoryName')] = '기타 아우터'

In [75]:
outer[outer['smallCategory']=="2s="]

,clothId,largeCategory,largeCategoryName,smallCategory,smallCategoryName,colorNo,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothSexCommon,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
11366,1532828,2,아우터,2s=,기타 아우터,23,에센셜 리니어 보아 후드 자켓 (FE2JKD6101XCRM),휠라,//image.msscdn.net/images/goods_img/20200730/1...,"79,000원",20200730,"['보아', '자켓', '플리스', '후드', '후리스', '캐리오버', '온라인전...",0,0,1,4.8,355,0,0,0,0,0,0,0,0,0
16581,1532833,2,아우터,2s=,기타 아우터,3,리니어 와펜 본딩 보아 자켓 (FE2JKC6102XGRY),휠라,//image.msscdn.net/images/goods_img/20200730/1...,"69,000원",20200730,"['보아', '자켓', '집업', '플리스', '후리스', '온라인전용']",0,0,1,4.6,483,0,0,0,0,0,0,0,0,0
29633,1532832,2,아우터,2s=,기타 아우터,2,우븐 포켓 보아 자켓 (FE2JKD6103XBLK),휠라,//image.msscdn.net/images/goods_img/20200730/1...,"89,000원",20200730,"['보아', '자켓', '집업', '플리스', '후리스', '캐리오버', '온라인전용']",0,1,0,4.6,483,0,0,0,0,0,0,0,0,0


In [76]:
outer.loc[outer.smallCategory=='2s=', ('smallCategory')] = '215'

In [77]:
outer['smallCategory'] = outer['smallCategory'].astype('int')

In [78]:
outer['smallCategory'].value_counts()

220    6148
203    5378
212    4926
206    4592
222    4118
215    4059
223    3310
221    3178
201    2253
217    2172
208    2057
202    1732
219    1587
218    1576
214    1457
207    1400
209     946
216     912
225     879
213     836
224     798
204     649
Name: smallCategory, dtype: int64

### smallCategory 원핫인코딩 변환

In [79]:
outer.insert(4, "smallCategoryOthers", outer['smallCategory'], True)
outer.insert(4, "smallCategoryJacket", outer['smallCategory'], True)
outer.insert(4, "smallCategoryVest", outer['smallCategory'], True)
outer.insert(4, "smallCategoryPaddedcoat", outer['smallCategory'], True)
outer.insert(4, "smallCategoryCoat", outer['smallCategory'], True)
outer.insert(4, "smallCategoryFleece", outer['smallCategory'], True)
outer.insert(4, "smallCategoryCardigan", outer['smallCategory'], True)
outer.insert(4, "smallCategoryMustang", outer['smallCategory'], True)
outer.insert(4, "smallCategoryRiders", outer['smallCategory'], True)
outer.insert(4, "smallCategoryBlouson", outer['smallCategory'], True)
outer.insert(4, "smallCategoryHoodie", outer['smallCategory'], True)

In [80]:
outer['smallCategoryHoodie'] = outer['smallCategoryHoodie'].apply(lambda x: 1 if x == 222 else 0)
outer['smallCategoryBlouson'] = outer['smallCategoryBlouson'].apply(lambda x: 1 if x == 201 else 0)
outer['smallCategoryRiders'] = outer['smallCategoryRiders'].apply(lambda x: 1 if x == 202 else 0)
outer['smallCategoryMustang'] = outer['smallCategoryMustang'].apply(lambda x: 1 if x == 225 else 0)
outer['smallCategoryCardigan'] = outer['smallCategoryCardigan'].apply(lambda x: 1 if x == 220 else 0)
outer['smallCategoryFleece'] = outer['smallCategoryFleece'].apply(lambda x: 1 if x == 223 else 0)
outer['smallCategoryCoat'] = outer['smallCategoryCoat'].apply(lambda x: 1 if x in [208, 207, 224, 209] else 0)
outer['smallCategoryPaddedcoat'] = outer['smallCategoryPaddedcoat'].apply(lambda x: 1 if x in [212, 213, 216] else 0)
outer['smallCategoryVest'] = outer['smallCategoryVest'].apply(lambda x: 1 if x == 221 else 0)
outer['smallCategoryJacket'] = outer['smallCategoryJacket'].apply(lambda x: 1 if x in [217, 203, 219, 218, 204, 214, 206] else 0)
outer['smallCategoryOthers'] = outer['smallCategoryOthers'].apply(lambda x: 1 if x == 215 else 0)

In [81]:
outer = outer.drop('smallCategory', axis=1)

In [82]:
outer.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategoryHoodie',
       'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang',
       'smallCategoryCardigan', 'smallCategoryFleece', 'smallCategoryCoat',
       'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket',
       'smallCategoryOthers', 'smallCategoryName', 'colorNo', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit',
       'feeling', 'stretch', 'visibility', 'thickness', 'seasonSpring',
       'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')

In [83]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 14):
    num += outer[outer.columns[i]].value_counts()[1]
    print(outer[outer.columns[i]].value_counts())
print(num)
print(outer.shape)

0    50845
1     4118
Name: smallCategoryHoodie, dtype: int64
0    52710
1     2253
Name: smallCategoryBlouson, dtype: int64
0    53231
1     1732
Name: smallCategoryRiders, dtype: int64
0    54084
1      879
Name: smallCategoryMustang, dtype: int64
0    48815
1     6148
Name: smallCategoryCardigan, dtype: int64
0    51653
1     3310
Name: smallCategoryFleece, dtype: int64
0    49762
1     5201
Name: smallCategoryCoat, dtype: int64
0    48289
1     6674
Name: smallCategoryPaddedcoat, dtype: int64
0    51785
1     3178
Name: smallCategoryVest, dtype: int64
0    37552
1    17411
Name: smallCategoryJacket, dtype: int64
0    50904
1     4059
Name: smallCategoryOthers, dtype: int64
54963
(54963, 36)


### clothSexCommon 변환

In [84]:
outer['clothSexMen'] = outer.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
outer['clothSexWomen'] = outer.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

In [85]:
outer = outer.drop('clothSexCommon', axis=1)

In [86]:
outer.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategoryHoodie',
       'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang',
       'smallCategoryCardigan', 'smallCategoryFleece', 'smallCategoryCoat',
       'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket',
       'smallCategoryOthers', 'smallCategoryName', 'colorNo', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [87]:
outer['clothSexMen'] = outer.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
outer['clothSexWomen'] = outer.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

In [88]:
print(outer['clothSexMen'].value_counts())
print(outer['clothSexWomen'].value_counts())
print(outer.shape)

1    34493
0    20470
Name: clothSexMen, dtype: int64
0    32168
1    22795
Name: clothSexWomen, dtype: int64
(54963, 35)


### color 원핫인코딩 변환

#### 아우터
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [89]:
outer.insert(15, "colorOthers", outer['colorNo'], True)
outer.insert(15, "colorPattern", outer['colorNo'], True)
outer.insert(15, "colorJean", outer['colorNo'], True)
outer.insert(15, "colorBeige", outer['colorNo'], True)
outer.insert(15, "colorBrown", outer['colorNo'], True)
outer.insert(15, "colorPurple", outer['colorNo'], True)
outer.insert(15, "colorBlue", outer['colorNo'], True)
outer.insert(15, "colorGreen", outer['colorNo'], True)
outer.insert(15, "colorYellow", outer['colorNo'], True)
outer.insert(15, "colorIvory", outer['colorNo'], True)
outer.insert(15, "colorOrange", outer['colorNo'], True)
outer.insert(15, "colorPink", outer['colorNo'], True)
outer.insert(15, "colorRed", outer['colorNo'], True)
outer.insert(15, "colorBlack", outer['colorNo'], True)
outer.insert(15, "colorGrey", outer['colorNo'], True)
outer.insert(15, "colorWhite", outer['colorNo'], True)

In [90]:
outer['colorWhite'] = outer['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
outer['colorGrey'] = outer['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
outer['colorBlack'] = outer['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
outer['colorRed'] = outer['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
outer['colorPink'] = outer['colorPink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
outer['colorOrange'] = outer['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
outer['colorIvory'] = outer['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
outer['colorYellow'] = outer['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
outer['colorGreen'] = outer['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
outer['colorBlue'] = outer['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
outer['colorPurple'] = outer['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
outer['colorBrown'] = outer['colorBrown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
outer['colorBeige'] = outer['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
outer['colorJean'] = outer['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
outer['colorPattern'] = outer['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
outer['colorOthers'] = outer['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [91]:
outer = outer.drop('colorNo', axis=1)

In [92]:
print(outer.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategoryHoodie',
       'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang',
       'smallCategoryCardigan', 'smallCategoryFleece', 'smallCategoryCoat',
       'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket',
       'smallCategoryOthers', 'smallCategoryName', 'colorWhite', 'colorGrey',
       'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory',
       'colorYellow', 'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown',
       'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')


In [93]:
# color도 모든 1의 값의 개수 == 행의 개수
num = 0
for i in range(15, 31):
    num+=outer[outer.columns[i]].value_counts()[1]
    print(outer[outer.columns[i]].value_counts())
print(num)
print(outer.shape)

0    53068
1     1895
Name: colorWhite, dtype: int64
0    48564
1     6399
Name: colorGrey, dtype: int64
0    38979
1    15984
Name: colorBlack, dtype: int64
0    54066
1      897
Name: colorRed, dtype: int64
0    53953
1     1010
Name: colorPink, dtype: int64
0    54538
1      425
Name: colorOrange, dtype: int64
0    51706
1     3257
Name: colorIvory, dtype: int64
0    54235
1      728
Name: colorYellow, dtype: int64
0    50135
1     4828
Name: colorGreen, dtype: int64
0    48743
1     6220
Name: colorBlue, dtype: int64
0    54257
1      706
Name: colorPurple, dtype: int64
0    52678
1     2285
Name: colorBrown, dtype: int64
0    48963
1     6000
Name: colorBeige, dtype: int64
0    54186
1      777
Name: colorJean, dtype: int64
0    54072
1      891
Name: colorPattern, dtype: int64
0    52302
1     2661
Name: colorOthers, dtype: int64
54963
(54963, 50)


In [94]:
outer.insert(31, "colorName", True)

In [95]:
outer['colorName'] = outer.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
outer['colorName'] = outer.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

In [96]:
outer['colorName'].value_counts()

black      15984
grey        6399
blue        6220
beige       6000
green       4828
ivory       3257
others      2661
brown       2285
white       1895
pink        1010
red          897
pattern      891
jean         777
yellow       728
purple       706
orange       425
Name: colorName, dtype: int64

### Price 전처리

In [97]:
outer['clothPrice'] = outer['clothPrice'].apply(lambda x: x.replace(",", ""))
outer['clothPrice'] = outer['clothPrice'].apply(lambda x: x.replace("원", ""))
outer['clothPrice']

0        109000
1         38700
2        159000
3        210000
4         95400
          ...  
54958    118300
54959    152100
54960    169200
54961    153000
54962    286000
Name: clothPrice, Length: 54963, dtype: object

In [98]:
outer['clothPrice'] = outer['clothPrice'].astype('int')

In [99]:
outer['clothReviewNo'] = outer['clothReviewNo'].apply(lambda x: x.replace(",", ""))
outer['clothReviewNo'] = outer['clothReviewNo'].astype('int')
outer['clothReviewNo']

0          0
1         16
2          0
3          0
4          0
        ... 
54958    424
54959    279
54960     37
54961      0
54962      0
Name: clothReviewNo, Length: 54963, dtype: int32

### Index를 NewGoodsNo로 변경

In [100]:
sub_outer = pd.read_json('./size_outer.json')
sub_outer

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,18798,L,187980097723,97723,"[{'text': '몸', 'value': 1}, {'text': '모양', 'va...",176.000000,68.000000,1.0,0.0,0,0,0,0,0,0,0,0,0,1,0,0
1,50937,S,5093700101241,101241,"[{'text': '가격', 'value': 2}, {'text': '옷', 'va...",167.500000,54.500000,0.5,0.5,0,0,0,2,0,0,0,0,1,0,0,1
2,50937,XL,5093700101242,101242,"[{'text': '가격', 'value': 2}, {'text': '옷', 'va...",171.000000,68.000000,1.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0
3,55782,38,5578200118983,118983,"[{'text': '옷', 'value': 2}, {'text': '가격', 'va...",171.500000,82.500000,1.0,0.0,1,0,0,0,0,0,0,0,0,0,1,0
4,55782,42,5578200118982,118982,"[{'text': '옷', 'value': 2}, {'text': '가격', 'va...",176.333333,80.666667,1.0,0.0,1,0,0,0,0,0,0,0,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52541,2408513,WHITE/FREE,240851385961,85961,"[{'text': '링클', 'value': 1}, {'text': '느낌', 'v...",176.000000,65.000000,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0
52542,2408513,WHITE/FREE,2408513110869,110869,"[{'text': '링클', 'value': 1}, {'text': '느낌', 'v...",182.000000,77.000000,1.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0
52543,2408664,DCH0_DARK-CHARCOL/095,2408664110810,110810,"[{'text': '디자인', 'value': 1}, {'text': '색감', '...",160.000000,60.000000,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0
52544,2410723,M,241072382324,82324,"[{'text': '개', 'value': 1}, {'text': '생각', 'va...",168.000000,54.000000,1.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0


In [101]:
outer = pd.merge(outer, sub_outer, how="left", left_on="clothId", right_on="goodsNo")
outer.shape

(85641, 72)

In [102]:
outer.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategoryHoodie',
       'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang',
       'smallCategoryCardigan', 'smallCategoryFleece', 'smallCategoryCoat',
       'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket',
       'smallCategoryOthers', 'smallCategoryName', 'colorWhite', 'colorGrey',
       'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory',
       'colorYellow', 'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown',
       'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'colorName',
       'clothName', 'brand', 'image', 'clothPrice', 'date', 'hashtags',
       'clothSexMen', 'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit',
       'feeling', 'stretch', 'visibility', 'thickness', 'seasonSpring',
       'seasonSummer', 'seasonFall', 'seasonWinter', 'goodsNo', 'goodsSize',
       'newGoodsNo', 'index', 'reviewNoun', 'userHeight', 'userWeight',
       'use

In [103]:
outer = outer.drop(["goodsNo", "index"], axis=1)
outer.shape

(85641, 70)

In [104]:
# 85641 : 반복문을 통해 변화된 skirt 행 개수
# 54963 : 원래의 skirt 행 개수
# 427 : 새로 생긴 skirt 행 개수
85641-54963

30678

In [105]:
len(sub_outer)

52546

In [106]:
len(sub_outer.goodsNo.unique())

21868

In [107]:
from copy import deepcopy
b = deepcopy(outer)

In [108]:
b.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategoryHoodie',
       'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang',
       'smallCategoryCardigan', 'smallCategoryFleece', 'smallCategoryCoat',
       'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket',
       'smallCategoryOthers', 'smallCategoryName', 'colorWhite', 'colorGrey',
       'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory',
       'colorYellow', 'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown',
       'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'colorName',
       'clothName', 'brand', 'image', 'clothPrice', 'date', 'hashtags',
       'clothSexMen', 'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit',
       'feeling', 'stretch', 'visibility', 'thickness', 'seasonSpring',
       'seasonSummer', 'seasonFall', 'seasonWinter', 'goodsSize', 'newGoodsNo',
       'reviewNoun', 'userHeight', 'userWeight', 'userSexMen', 'userSexWomen',
 

In [109]:
col_list_1 = ['newGoodsNo', 'userSexMen', 'userSexWomen']
col_list_2 = ['goodsSize', 'reviewNoun', 'userHeight', 'userWeight', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12']
check_list_1 = ['clothId',  'clothSexMen', 'clothSexWomen']
check_list_2 = ['Free', [], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(len(col_list_1)):
    b[col_list_1[i]] = b[col_list_1[i]].fillna(0)
    b[col_list_1[i]] = b.apply(lambda x: x[check_list_1[i]] if x[col_list_1[i]]==0 else x[col_list_1[i]], axis=1)
    if i==0:
        b[col_list_1[i]] = b[col_list_1[i]].astype('int64')
    
for i in range(len(col_list_2)):
    b[col_list_2[i]] = b[col_list_2[i]].fillna(0)
    b[col_list_2[i]] = b.apply(lambda x: check_list_2[i] if x[col_list_2[i]]==0 else x[col_list_2[i]], axis=1)
    if i == 0:
        b[col_list_2[i]] = b[col_list_2[i]].astype('str')
    elif i >= 3:
        b[col_list_2[i]] = b[col_list_2[i]].astype('int64')

In [110]:
outer = deepcopy(b)

In [111]:
# 이미지 null 값 확인
print(len(outer[outer.image.isnull()]))

0


In [112]:
# hashtags 전처리
outer.hashtags = outer.apply(lambda x: x.hashtags.replace("['", "").replace("']", "").replace("'", "").replace(" ", "").split(","), axis=1)

In [113]:
outer.userHeight = outer.userHeight.astype('float')
outer.userWeight = outer.userWeight.astype('float')
outer.userSexMen = outer.userSexMen.astype('float')
outer.userSexWomen = outer.userSexWomen.astype('float')

In [114]:
outer.reviewNoun.iloc[2]

[]

In [115]:
outer.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategoryHoodie',
       'smallCategoryBlouson', 'smallCategoryRiders', 'smallCategoryMustang',
       'smallCategoryCardigan', 'smallCategoryFleece', 'smallCategoryCoat',
       'smallCategoryPaddedcoat', 'smallCategoryVest', 'smallCategoryJacket',
       'smallCategoryOthers', 'smallCategoryName', 'colorWhite', 'colorGrey',
       'colorBlack', 'colorRed', 'colorPink', 'colorOrange', 'colorIvory',
       'colorYellow', 'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown',
       'colorBeige', 'colorJean', 'colorPattern', 'colorOthers', 'colorName',
       'clothName', 'brand', 'image', 'clothPrice', 'date', 'hashtags',
       'clothSexMen', 'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit',
       'feeling', 'stretch', 'visibility', 'thickness', 'seasonSpring',
       'seasonSummer', 'seasonFall', 'seasonWinter', 'goodsSize', 'newGoodsNo',
       'reviewNoun', 'userHeight', 'userWeight', 'userSexMen', 'userSexWomen',
 

In [116]:
outer.columns = ['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHoodie', 'smallCategoryBlouson', 'smallCategoryRiders',
       'smallCategoryMustang', 'smallCategoryCardigan', 'smallCategoryFleece',
       'smallCategoryCoat', 'smallCategoryPaddedcoat', 'smallCategoryVest',
       'smallCategoryJacket', 'smallCategoryOthers',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'clothImg', 'clothPrice', 'date', 'clothHashtags', 'clothMale',
       'clothFemale', 'clothRate', 'clothReviewCnt', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'newClothId', 'reviewNoun', 'userHeight',
       'userWeight', 'userMale', 'userFemale', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12']

In [117]:
outer = outer[['newClothId', 'clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryHoodie', 'smallCategoryBlouson', 'smallCategoryRiders',
       'smallCategoryMustang', 'smallCategoryCardigan', 'smallCategoryFleece',
       'smallCategoryCoat', 'smallCategoryPaddedcoat', 'smallCategoryVest',
       'smallCategoryJacket', 'smallCategoryOthers',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'clothImg', 'clothPrice', 'date', 'clothHashtags', 'clothMale',
       'clothFemale', 'clothRate', 'clothReviewCnt', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'reviewNoun', 'userHeight',
       'userWeight', 'userMale', 'userFemale', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12']]

In [292]:
outer.to_json('final_outer.json', orient='records', force_ascii=False)

In [119]:
outer.to_csv('final_outer.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

# Pants

In [120]:
pants = pd.read_csv('./musinsa_clothes_pants.csv')

In [121]:
pants.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [122]:
pants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57679 entries, 0 to 57678
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            57679 non-null  int64  
 1   largeCategory      57679 non-null  int64  
 2   largeCategoryName  57679 non-null  object 
 3   smallCategory      57679 non-null  int64  
 4   smallCategoryName  57679 non-null  object 
 5   colorNo            57679 non-null  int64  
 6   clothName          57679 non-null  object 
 7   brand              57679 non-null  object 
 8   image              57679 non-null  object 
 9   clothPrice         57679 non-null  object 
 10  date               57679 non-null  int64  
 11  hashtags           57679 non-null  object 
 12  clothSexMen        57679 non-null  int64  
 13  clothSexWomen      57679 non-null  int64  
 14  clothSexCommon     57679 non-null  int64  
 15  clothRate          57679 non-null  float64
 16  clothReviewNo      576

In [123]:
pants.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,5.767900e+04,57679.0,57679.000000,57679.000000,5.767900e+04,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000,57679.000000
mean,1.840605e+06,3.0,305.918601,15.812115,2.020691e+07,0.645868,0.324295,0.029768,2.475041,1.118345,0.775482,0.989962,1.485844,0.960037,0.248669,0.183620,0.251443,0.175038
std,4.652932e+05,0.0,2.625236,16.485140,1.145007e+04,0.478254,0.468115,0.169949,2.367970,1.739613,1.266744,1.657343,2.221116,1.478658,0.432245,0.387177,0.433847,0.380003
min,2.586800e+04,3.0,302.000000,1.000000,2.012110e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.564060e+06,3.0,304.000000,2.000000,2.020083e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.945703e+06,3.0,306.000000,7.000000,2.021051e+07,1.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.204482e+06,3.0,308.000000,25.000000,2.021103e+07,1.000000,1.000000,0.000000,4.800000,3.000000,1.000000,2.000000,5.000000,3.000000,0.000000,0.000000,1.000000,0.000000
max,2.413615e+06,3.0,310.000000,71.000000,2.022031e+07,1.000000,1.000000,1.000000,5.000000,5.000000,6.000000,6.000000,6.000000,5.000000,1.000000,1.000000,1.000000,1.000000


In [124]:
pants = pants.drop_duplicates(subset = "clothId")
pants = pants.reset_index(drop=True)

In [125]:
pants['smallCategory'] = pants['smallCategory'].astype('str')

In [126]:
pants['largeCategory'].unique()

array([3], dtype=int64)

In [127]:
pants['largeCategoryName'].unique()

array(['바지'], dtype=object)

In [128]:
pants['smallCategory'].unique()

array(['306', '304', '308', '309', '302', '310', '307', '305'],
      dtype=object)

In [129]:
pants['smallCategoryName'].unique()

array(['기타 바지', '트레이닝/조거 팬츠', '슈트 팬츠/슬랙스', '숏 팬츠', '데님 팬츠', '기능성 하의',
       '점프 슈트/오버올', '코튼 팬츠', '레깅스'], dtype=object)

In [130]:
pants[pants['smallCategoryName']=='기능성 하의']

,clothId,largeCategory,largeCategoryName,smallCategory,smallCategoryName,colorNo,clothName,brand,image,clothPrice,date,hashtags,clothSexMen,clothSexWomen,clothSexCommon,clothRate,clothReviewNo,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
32,1321523,3,바지,310,기능성 하의,3,여성 요가복 DEVI-B0036-그레이 필라테스 레깅스 9부 베이직,데비웨어,//image.msscdn.net/images/goods_img/20200225/1...,"24,000원",20200225,"['여성요가복', '요가복', '필라테스', '필라테스복', '필라테스운동복', '...",0,1,0,0.0,0,0,0,0,0,0,0,0,0,0
34,1667376,3,바지,310,기능성 하의,3,여성 사이드 스타프린트 레깅스 GREY / FF0WLG01F87,피피라핀,//image.msscdn.net/images/goods_img/20201028/1...,"29,900원",20201028,"['레깅스', '여성레깅스', '겨울레깅스', '여자레깅스', '케주얼레깅스']",0,1,0,4.0,2,1,3,5,5,3,1,0,1,1
138,2316809,3,바지,310,기능성 하의,3,기모_CREORA & CELLA LEGGINS GR,아이코노그라피,//image.msscdn.net/images/goods_img/20220120/2...,"19,800원",20220120,"['레깅스추천', '레깅스', '여성레깅스', '레깅스컬렉션', '레깅스팬츠', '...",0,1,0,0.0,0,1,1,5,5,3,1,0,1,1
146,2185722,3,바지,310,기능성 하의,3,RIF (리프) 여성 본딩 타이즈 팬츠_Ciment,아이더,//image.msscdn.net/images/goods_img/20211019/2...,"129,000원",20211019,[],0,1,0,0.0,0,0,0,0,0,0,0,0,0,0
161,2301479,3,바지,310,기능성 하의,3,TS 럭스 하이라이즈 레깅스 - 그레이 / HB2028,리복,//image.msscdn.net/images/goods_img/20220111/2...,"44,900원",20220111,"['타이츠', '하이웨스트레깅스', '하이웨스트타이츠', '요가', '필라테스', ...",0,1,0,0.0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57567,1161106,3,바지,310,기능성 하의,15,여성 사티 팬츠(하의) 4종 시리즈,69슬램,//image.msscdn.net/images/goods_img/20190920/1...,"69,000원",20190920,"['래쉬팬츠', '레깅스', '엑티브웨어', '여성요가복하의', '여성필라테스하의']",0,1,0,0.0,0,0,0,0,0,0,0,0,0,0
57579,876594,3,바지,310,기능성 하의,15,액티브웨어 레깅스 센느 네이비,밸롭,//image.msscdn.net/images/goods_img/20181010/8...,"39,000원",20181010,[],0,1,0,0.0,0,0,0,0,0,0,0,0,0,0
57583,1614139,3,바지,310,기능성 하의,15,CLWP7004 7부 팬츠 마젠타,씨엘코코,//image.msscdn.net/images/goods_img/20200921/1...,"13,900원",20200921,"['요가복', '필라테스복', '레깅스', '요가복상의', '요가복하의', '요가복...",0,1,0,0.0,0,0,0,0,0,0,0,0,0,0
57592,1441580,3,바지,310,기능성 하의,15,AW01PL04 아이스 숏 레깅스_다크 그레이,애스플래폼,//image.msscdn.net/images/goods_img/20200512/1...,"39,000원",20200512,"['그레이레깅스', '레깅스반바지', '반바지레깅스', '숏레깅스', '아이스레깅스...",0,1,0,0.0,0,1,1,5,2,2,0,1,0,0


In [131]:
pants.loc[pants.smallCategoryName=='기능성 하의', ('smallCategoryName')] = '레깅스'
pants.loc[pants.smallCategoryName=='기능성 하의', ('smallCategory')] = '305'

In [132]:
pants['smallCategory'] = pants['smallCategory'].astype('int')

### smallCategory 원핫인코딩 변환

In [133]:
pants.insert(4, "smallCategoryOthers", pants['smallCategory'], True)
pants.insert(4, "smallCategoryJumpsuit", pants['smallCategory'], True)
pants.insert(4, "smallCategoryLeggings", pants['smallCategory'], True)
pants.insert(4, "smallCategoryShortpants", pants['smallCategory'], True)
pants.insert(4, "smallCategoryJoggerpants", pants['smallCategory'], True)
pants.insert(4, "smallCategorySlacks", pants['smallCategory'], True)
pants.insert(4, "smallCategoryCottonpants", pants['smallCategory'], True)
pants.insert(4, "smallCategoryDenimpants", pants['smallCategory'], True)

In [134]:
pants['smallCategoryDenimpants'] = pants['smallCategoryDenimpants'].apply(lambda x: 1 if x == 302 else 0)
pants['smallCategoryCottonpants'] = pants['smallCategoryCottonpants'].apply(lambda x: 1 if x == 307 else 0)
pants['smallCategorySlacks'] = pants['smallCategorySlacks'].apply(lambda x: 1 if x == 308 else 0)
pants['smallCategoryJoggerpants'] = pants['smallCategoryJoggerpants'].apply(lambda x: 1 if x == 304 else 0)
pants['smallCategoryShortpants'] = pants['smallCategoryShortpants'].apply(lambda x: 1 if x == 309 else 0)
pants['smallCategoryLeggings'] = pants['smallCategoryLeggings'].apply(lambda x: 1 if x == 305 else 0)
pants['smallCategoryJumpsuit'] = pants['smallCategoryJumpsuit'].apply(lambda x: 1 if x == 310 else 0)
pants['smallCategoryOthers'] = pants['smallCategoryOthers'].apply(lambda x: 1 if x == 306 else 0)

In [135]:
pants = pants.drop('smallCategory', axis=1)

In [136]:
pants.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryDenimpants', 'smallCategoryCottonpants',
       'smallCategorySlacks', 'smallCategoryJoggerpants',
       'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothSexCommon',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [137]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 11):
    num += pants[pants.columns[i]].value_counts()[1]
    print(pants[pants.columns[i]].value_counts())
print(num)
print(pants.shape)

0    47646
1     9954
Name: smallCategoryDenimpants, dtype: int64
0    49928
1     7672
Name: smallCategoryCottonpants, dtype: int64
0    50396
1     7204
Name: smallCategorySlacks, dtype: int64
0    42486
1    15114
Name: smallCategoryJoggerpants, dtype: int64
0    47026
1    10574
Name: smallCategoryShortpants, dtype: int64
0    57578
1       22
Name: smallCategoryLeggings, dtype: int64
0    54730
1     2870
Name: smallCategoryJumpsuit, dtype: int64
0    53410
1     4190
Name: smallCategoryOthers, dtype: int64
57600
(57600, 33)


### clothSexCommon 변환

In [138]:
pants['clothSexMen'] = pants.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
pants['clothSexWomen'] = pants.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

In [139]:
pants = pants.drop('clothSexCommon', axis=1)

In [140]:
pants.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryDenimpants', 'smallCategoryCottonpants',
       'smallCategorySlacks', 'smallCategoryJoggerpants',
       'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers', 'smallCategoryName',
       'colorNo', 'clothName', 'brand', 'image', 'clothPrice', 'date',
       'hashtags', 'clothSexMen', 'clothSexWomen', 'clothRate',
       'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility', 'thickness',
       'seasonSpring', 'seasonSummer', 'seasonFall', 'seasonWinter'],
      dtype='object')

In [141]:
pants['clothSexMen'] = pants.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
pants['clothSexWomen'] = pants.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

In [142]:
print(pants['clothSexMen'].value_counts())
print(pants['clothSexWomen'].value_counts())
print(pants.shape)

1    38914
0    18686
Name: clothSexMen, dtype: int64
0    37198
1    20402
Name: clothSexWomen, dtype: int64
(57600, 32)


### color 원핫인코딩 변환

#### 바지
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [143]:
pants.insert(13, "colorOthers", pants['colorNo'], True)
pants.insert(13, "colorPattern", pants['colorNo'], True)
pants.insert(13, "colorJean", pants['colorNo'], True)
pants.insert(13, "colorBeige", pants['colorNo'], True)
pants.insert(13, "colorBrown", pants['colorNo'], True)
pants.insert(13, "colorPurple", pants['colorNo'], True)
pants.insert(13, "colorBlue", pants['colorNo'], True)
pants.insert(13, "colorGreen", pants['colorNo'], True)
pants.insert(13, "colorYellow", pants['colorNo'], True)
pants.insert(13, "colorIvory", pants['colorNo'], True)
pants.insert(13, "colorOrange", pants['colorNo'], True)
pants.insert(13, "colorPink", pants['colorNo'], True)
pants.insert(13, "colorRed", pants['colorNo'], True)
pants.insert(13, "colorBlack", pants['colorNo'], True)
pants.insert(13, "colorGrey", pants['colorNo'], True)
pants.insert(13, "colorWhite", pants['colorNo'], True)

In [144]:
pants['colorWhite'] = pants['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
pants['colorGrey'] = pants['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
pants['colorBlack'] = pants['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
pants['colorRed'] = pants['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
pants['colorPink'] = pants['colorPink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
pants['colorOrange'] = pants['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
pants['colorIvory'] = pants['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
pants['colorYellow'] = pants['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
pants['colorGreen'] = pants['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
pants['colorBlue'] = pants['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
pants['colorPurple'] = pants['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
pants['colorBrown'] = pants['colorBrown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
pants['colorBeige'] = pants['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
pants['colorJean'] = pants['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
pants['colorPattern'] = pants['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 69, 66] else 0)
pants['colorOthers'] = pants['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [145]:
print(pants.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryDenimpants', 'smallCategoryCottonpants',
       'smallCategorySlacks', 'smallCategoryJoggerpants',
       'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers', 'smallCategoryName',
       'colorNo', 'colorWhite', 'colorGrey', 'colorBlack', 'colorRed',
       'colorPink', 'colorOrange', 'colorIvory', 'colorYellow', 'colorGreen',
       'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige', 'colorJean',
       'colorPattern', 'colorOthers', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')


In [146]:
pants = pants.drop('colorNo', axis=1)

In [147]:
# color도 모든 1의 값의 개수 == 행의 개수
num = 0
for i in range(12, 28):
    num += pants[pants.columns[i]].value_counts()[1]
    print(pants[pants.columns[i]].value_counts())
print(num)
print(pants.shape)

0    55827
1     1773
Name: colorWhite, dtype: int64
0    49481
1     8119
Name: colorGrey, dtype: int64
0    42691
1    14909
Name: colorBlack, dtype: int64
0    56943
1      657
Name: colorRed, dtype: int64
0    56783
1      817
Name: colorPink, dtype: int64
0    57163
1      437
Name: colorOrange, dtype: int64
0    54887
1     2713
Name: colorIvory, dtype: int64
0    57034
1      566
Name: colorYellow, dtype: int64
0    53309
1     4291
Name: colorGreen, dtype: int64
0    51080
1     6520
Name: colorBlue, dtype: int64
0    56953
1      647
Name: colorPurple, dtype: int64
0    55964
1     1636
Name: colorBrown, dtype: int64
0    52433
1     5167
Name: colorBeige, dtype: int64
0    51267
1     6333
Name: colorJean, dtype: int64
0    57187
1      413
Name: colorPattern, dtype: int64
0    54998
1     2602
Name: colorOthers, dtype: int64
57600
(57600, 47)


In [148]:
pants.insert(28, "colorName", True)

In [149]:
pants['colorName'] = pants.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
pants['colorName'] = pants.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

In [150]:
pants['colorName'].value_counts()

black      14909
grey        8119
blue        6520
jean        6333
beige       5167
green       4291
ivory       2713
others      2602
white       1773
brown       1636
pink         817
red          657
purple       647
yellow       566
orange       437
pattern      413
Name: colorName, dtype: int64

### Price 전처리

In [151]:
pants['clothPrice'] = pants['clothPrice'].apply(lambda x: x.replace(",", ""))
pants['clothPrice'] = pants['clothPrice'].apply(lambda x: x.replace("원", ""))
pants['clothPrice']

0        130000
1         69000
2        199000
3         95000
4         69000
          ...  
57595    149000
57596     70500
57597     69000
57598    110000
57599    171000
Name: clothPrice, Length: 57600, dtype: object

In [152]:
pants['clothPrice'] = pants['clothPrice'].astype('int')

In [153]:
pants['clothReviewNo'] = pants['clothReviewNo'].apply(lambda x: x.replace(",", ""))
pants['clothReviewNo'] = pants['clothReviewNo'].astype('int')
pants['clothReviewNo']

0        0
1        0
2        0
3        0
4        0
        ..
57595    0
57596    0
57597    0
57598    0
57599    0
Name: clothReviewNo, Length: 57600, dtype: int32

### Index를 NewGoodsNo로 변경

In [154]:
sub_pants = pd.read_json('./size_pants.json')
sub_pants

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,25868,28/28,258680052215,52215,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",167.833333,54.333333,0.666667,0.333333,0,0,1,1,0,0,0,0,1,2,1,0
1,25868,28/30,258680052218,52218,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",174.000000,53.000000,1.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0
2,25868,28/32,258680052204,52204,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",167.000000,50.000000,0.000000,1.000000,0,0,0,0,0,0,0,0,1,0,0,0
3,25868,29/28,258680052205,52205,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",170.222222,63.444444,1.000000,0.000000,0,0,2,0,1,0,1,0,1,2,1,1
4,25868,29/30,258680052201,52201,"[{'text': '사이즈', 'value': 32}, {'text': '핏', '...",174.500000,62.500000,1.000000,0.000000,0,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74132,2407517,30,240751752980,52980,"[{'text': '노', 'value': 1}, {'text': '베이지색', '...",172.000000,65.000000,1.000000,0.000000,0,0,1,0,0,0,0,0,0,0,0,0
74133,2407719,L,240771913225,13225,"[{'text': '만족', 'value': 1}, {'text': '지퍼', 'v...",183.000000,78.000000,1.000000,0.000000,0,0,1,0,0,0,0,0,0,0,0,0
74134,2410780,L,241078053057,53057,"[{'text': '재질', 'value': 1}, {'text': '생각', 'v...",169.000000,56.000000,0.000000,1.000000,0,0,1,0,0,0,0,0,0,0,0,0
74135,2411470,그레이/L(75),241147046647,46647,"[{'text': '핏', 'value': 1}, {'text': '평소', 'va...",167.000000,59.000000,0.000000,1.000000,0,0,1,0,0,0,0,0,0,0,0,0


In [155]:
pants = pd.merge(pants, sub_pants, how="left", left_on="clothId", right_on="goodsNo")
pants.shape

(106878, 69)

In [156]:
pants.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryDenimpants', 'smallCategoryCottonpants',
       'smallCategorySlacks', 'smallCategoryJoggerpants',
       'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers', 'smallCategoryName',
       'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink',
       'colorOrange', 'colorIvory', 'colorYellow', 'colorGreen', 'colorBlue',
       'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern',
       'colorOthers', 'colorName', 'clothName', 'brand', 'image', 'clothPrice',
       'date', 'hashtags', 'clothSexMen', 'clothSexWomen', 'clothRate',
       'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility', 'thickness',
       'seasonSpring', 'seasonSummer', 'seasonFall', 'seasonWinter', 'goodsNo',
       'goodsSize', 'newGoodsNo', 'index', 'reviewNoun', 'userHeight',
       'userWeight', 'userSexMen', 'userSexWomen', 'month1', 'month2',
     

In [157]:
pants = pants.drop(["goodsNo", "index"], axis=1)
pants.shape

(106878, 67)

In [158]:
# 106879 : 반복문을 통해 변화된 skirt 행 개수
# 57600 : 원래의 skirt 행 개수
# 49279 : 새로 생긴 skirt 행 개수
106879-57600

49279

In [159]:
len(sub_pants)

74137

In [160]:
len(sub_pants.goodsNo.unique())

24859

In [161]:
from copy import deepcopy
b = deepcopy(pants)

In [162]:
b.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryDenimpants', 'smallCategoryCottonpants',
       'smallCategorySlacks', 'smallCategoryJoggerpants',
       'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers', 'smallCategoryName',
       'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink',
       'colorOrange', 'colorIvory', 'colorYellow', 'colorGreen', 'colorBlue',
       'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern',
       'colorOthers', 'colorName', 'clothName', 'brand', 'image', 'clothPrice',
       'date', 'hashtags', 'clothSexMen', 'clothSexWomen', 'clothRate',
       'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility', 'thickness',
       'seasonSpring', 'seasonSummer', 'seasonFall', 'seasonWinter',
       'goodsSize', 'newGoodsNo', 'reviewNoun', 'userHeight', 'userWeight',
       'userSexMen', 'userSexWomen', 'month1', 'month2', 'month3', 'month4',
     

In [163]:
col_list_1 = ['newGoodsNo', 'userSexMen', 'userSexWomen']
col_list_2 = ['goodsSize', 'reviewNoun', 'userHeight', 'userWeight', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12']
check_list_1 = ['clothId',  'clothSexMen', 'clothSexWomen']
check_list_2 = ['Free', [], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(len(col_list_1)):
    b[col_list_1[i]] = b[col_list_1[i]].fillna(0)
    b[col_list_1[i]] = b.apply(lambda x: x[check_list_1[i]] if x[col_list_1[i]]==0 else x[col_list_1[i]], axis=1)
    if i==0:
        b[col_list_1[i]] = b[col_list_1[i]].astype('int64')
    
for i in range(len(col_list_2)):
    b[col_list_2[i]] = b[col_list_2[i]].fillna(0)
    b[col_list_2[i]] = b.apply(lambda x: check_list_2[i] if x[col_list_2[i]]==0 else x[col_list_2[i]], axis=1)
    if i == 0:
        b[col_list_2[i]] = b[col_list_2[i]].astype('str')
    elif i >= 3:
        b[col_list_2[i]] = b[col_list_2[i]].astype('int64')

In [164]:
pants = deepcopy(b)

In [165]:
# 이미지 null 값 확인
print(len(pants[pants.image.isnull()]))

0


In [166]:
# hashtags 전처리
pants.hashtags = pants.apply(lambda x: x.hashtags.replace("['", "").replace("']", "").replace("'", "").replace(" ", "").split(","), axis=1)

In [167]:
pants.userHeight = pants.userHeight.astype('float')
pants.userWeight = pants.userWeight.astype('float')
pants.userSexMen = pants.userSexMen.astype('float')
pants.userSexWomen = pants.userSexWomen.astype('float')

In [168]:
pants.reviewNoun[5]

[{'text': '하비', 'value': 1},
 {'text': '허벅지', 'value': 1},
 {'text': '길이', 'value': 1},
 {'text': '통', 'value': 1},
 {'text': '자체', 'value': 1},
 {'text': '타이', 'value': 1},
 {'text': '트', 'value': 1},
 {'text': '살', 'value': 1},
 {'text': '부각', 'value': 1},
 {'text': '무지', 'value': 1},
 {'text': '티', 'value': 1},
 {'text': '다른', 'value': 1},
 {'text': '색도', 'value': 1},
 {'text': '진짜', 'value': 1},
 {'text': '반바지', 'value': 1},
 {'text': '만', 'value': 1},
 {'text': '살이', 'value': 1},
 {'text': '좀', 'value': 1},
 {'text': '자주', 'value': 1},
 {'text': '뉴', 'value': 1},
 {'text': '정말', 'value': 1},
 {'text': '여름', 'value': 1}]

In [169]:
pants.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryDenimpants', 'smallCategoryCottonpants',
       'smallCategorySlacks', 'smallCategoryJoggerpants',
       'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers', 'smallCategoryName',
       'colorWhite', 'colorGrey', 'colorBlack', 'colorRed', 'colorPink',
       'colorOrange', 'colorIvory', 'colorYellow', 'colorGreen', 'colorBlue',
       'colorPurple', 'colorBrown', 'colorBeige', 'colorJean', 'colorPattern',
       'colorOthers', 'colorName', 'clothName', 'brand', 'image', 'clothPrice',
       'date', 'hashtags', 'clothSexMen', 'clothSexWomen', 'clothRate',
       'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility', 'thickness',
       'seasonSpring', 'seasonSummer', 'seasonFall', 'seasonWinter',
       'goodsSize', 'newGoodsNo', 'reviewNoun', 'userHeight', 'userWeight',
       'userSexMen', 'userSexWomen', 'month1', 'month2', 'month3', 'month4',
     

In [170]:
pants.columns = ['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryDenimpants', 'smallCategoryCottonpants', 'smallCategorySlacks',
       'smallCategoryJoggerpants', 'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'clothImg', 'clothPrice', 'date', 'clothHashtags', 'clothMale',
       'clothFemale', 'clothRate', 'clothReviewCnt', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'newClothId', 'reviewNoun', 'userHeight',
       'userWeight', 'userMale', 'userFemale', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12']

In [171]:
pants = pants[['newClothId', 'clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryDenimpants', 'smallCategoryCottonpants', 'smallCategorySlacks',
       'smallCategoryJoggerpants', 'smallCategoryShortpants', 'smallCategoryLeggings',
       'smallCategoryJumpsuit', 'smallCategoryOthers',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'clothImg', 'clothPrice', 'date', 'clothHashtags', 'clothMale',
       'clothFemale', 'clothRate', 'clothReviewCnt', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'reviewNoun', 'userHeight',
       'userWeight', 'userMale', 'userFemale', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12']]

In [172]:
pants.to_json('final_pants.json', orient='records', force_ascii=False)

In [173]:
pants.to_csv('final_pants.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

# Onepiece

In [174]:
onepiece = pd.read_csv('./musinsa_clothes_onepiece.csv')

In [175]:
onepiece.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [176]:
onepiece.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9456 entries, 0 to 9455
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            9456 non-null   int64  
 1   largeCategory      9456 non-null   int64  
 2   largeCategoryName  9456 non-null   object 
 3   smallCategory      9456 non-null   int64  
 4   smallCategoryName  9456 non-null   object 
 5   colorNo            9456 non-null   int64  
 6   clothName          9456 non-null   object 
 7   brand              9456 non-null   object 
 8   image              9456 non-null   object 
 9   clothPrice         9456 non-null   object 
 10  date               9456 non-null   int64  
 11  hashtags           9456 non-null   object 
 12  clothSexMen        9456 non-null   int64  
 13  clothSexWomen      9456 non-null   int64  
 14  clothSexCommon     9456 non-null   int64  
 15  clothRate          9456 non-null   float64
 16  clothReviewNo      9456 

In [177]:
onepiece.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,9.456000e+03,9456.0,9456.000000,9456.000000,9.456000e+03,9456.0,9456.0,9456.0,9456.000000,9456.000000,9456.000000,9456.000000,9456.000000,9456.000000,9456.000000,9456.000000,9456.000000,9456.000000
mean,1.828642e+06,4.0,407.105859,16.600465,2.020675e+07,0.0,1.0,0.0,1.104209,1.119924,0.748519,0.949979,1.498731,0.940567,0.240376,0.202728,0.218591,0.118655
std,4.241448e+05,0.0,0.776100,17.107649,9.750778e+03,0.0,0.0,0.0,1.993787,1.610713,1.159970,1.539927,2.121980,1.386594,0.427335,0.402053,0.413313,0.323399
min,1.374480e+05,4.0,406.000000,1.000000,2.014103e+07,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.509934e+06,4.0,406.000000,2.000000,2.020071e+07,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.942870e+06,4.0,407.000000,8.000000,2.021051e+07,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.143857e+06,4.0,408.000000,30.000000,2.021092e+07,0.0,1.0,0.0,0.000000,3.000000,1.000000,1.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000
max,2.413328e+06,4.0,408.000000,71.000000,2.022031e+07,0.0,1.0,0.0,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.000000,1.000000,1.000000,1.000000


In [178]:
onepiece = onepiece.drop_duplicates(subset = "clothId")
onepiece = onepiece.reset_index(drop=True)

In [179]:
onepiece['smallCategory'] = onepiece['smallCategory'].astype('str')

In [180]:
onepiece['largeCategory'].unique()

array([4], dtype=int64)

In [181]:
onepiece['largeCategoryName'].unique()

array(['원피스'], dtype=object)

In [182]:
onepiece['smallCategory'].unique()

array(['407', '408', '406'], dtype=object)

In [183]:
onepiece['smallCategoryName'].unique()

array(['미디 원피스', '맥시 원피스', '미니 원피스'], dtype=object)

In [184]:
onepiece['smallCategory'] = onepiece['smallCategory'].astype('int')

### smallCategory 원핫인코딩 변환

In [185]:
onepiece.insert(4, "smallCategoryMaxidress", onepiece['smallCategory'], True)
onepiece.insert(4, "smallCategoryMidi", onepiece['smallCategory'], True)
onepiece.insert(4, "smallCategoryMinidress", onepiece['smallCategory'], True)

In [186]:
onepiece['smallCategoryMinidress'] = onepiece['smallCategoryMinidress'].apply(lambda x: 1 if x == 406 else 0)
onepiece['smallCategoryMidi'] = onepiece['smallCategoryMidi'].apply(lambda x: 1 if x == 407 else 0)
onepiece['smallCategoryMaxidress'] = onepiece['smallCategoryMaxidress'].apply(lambda x: 1 if x == 408 else 0)

In [187]:
onepiece = onepiece.drop('smallCategory', axis=1)

In [188]:
onepiece.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [189]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 6):
    num += onepiece[onepiece.columns[i]].value_counts()[1]
    print(onepiece[onepiece.columns[i]].value_counts())
print(num)
print(onepiece.shape)

0    7054
1    2394
Name: smallCategoryMinidress, dtype: int64
0    5793
1    3655
Name: smallCategoryMidi, dtype: int64
0    6049
1    3399
Name: smallCategoryMaxidress, dtype: int64
9448
(9448, 28)


### clothSexCommon 변환

In [190]:
onepiece['clothSexMen'] = onepiece.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
onepiece['clothSexWomen'] = onepiece.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

In [191]:
onepiece = onepiece.drop('clothSexCommon', axis=1)

In [192]:
onepiece.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [193]:
onepiece['clothSexMen'] = onepiece.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
onepiece['clothSexWomen'] = onepiece.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

In [194]:
print(onepiece['clothSexMen'].value_counts())
print(onepiece['clothSexWomen'].value_counts())
print(onepiece.shape)

0    9448
Name: clothSexMen, dtype: int64
1    9448
Name: clothSexWomen, dtype: int64
(9448, 27)


### color 원핫인코딩 변환

#### 원피스
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (50, 47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 56, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [195]:
onepiece.insert(8, "colorOthers", onepiece['colorNo'], True)
onepiece.insert(8, "colorPattern", onepiece['colorNo'], True)
onepiece.insert(8, "colorJean", onepiece['colorNo'], True)
onepiece.insert(8, "colorBeige", onepiece['colorNo'], True)
onepiece.insert(8, "colorBrown", onepiece['colorNo'], True)
onepiece.insert(8, "colorPurple", onepiece['colorNo'], True)
onepiece.insert(8, "colorBlue", onepiece['colorNo'], True)
onepiece.insert(8, "colorGreen", onepiece['colorNo'], True)
onepiece.insert(8, "colorYellow", onepiece['colorNo'], True)
onepiece.insert(8, "colorIvory", onepiece['colorNo'], True)
onepiece.insert(8, "colorOrange", onepiece['colorNo'], True)
onepiece.insert(8, "colorPink", onepiece['colorNo'], True)
onepiece.insert(8, "colorRed", onepiece['colorNo'], True)
onepiece.insert(8, "colorBlack", onepiece['colorNo'], True)
onepiece.insert(8, "colorGrey", onepiece['colorNo'], True)
onepiece.insert(8, "colorWhite", onepiece['colorNo'], True)

In [196]:
onepiece['colorWhite'] = onepiece['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
onepiece['colorGrey'] = onepiece['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
onepiece['colorBlack'] = onepiece['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
onepiece['colorRed'] = onepiece['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
onepiece['colorPink'] = onepiece['colorPink'].apply(lambda x: 1 if x in [50, 47, 10, 45, 48, 55] else 0)
onepiece['colorOrange'] = onepiece['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
onepiece['colorIvory'] = onepiece['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
onepiece['colorYellow'] = onepiece['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
onepiece['colorGreen'] = onepiece['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
onepiece['colorBlue'] = onepiece['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
onepiece['colorPurple'] = onepiece['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
onepiece['colorBrown'] = onepiece['colorBrown'].apply(lambda x: 1 if x in [4, 56, 27] else 0)
onepiece['colorBeige'] = onepiece['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
onepiece['colorJean'] = onepiece['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
onepiece['colorPattern'] = onepiece['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 64, 69, 66] else 0)
onepiece['colorOthers'] = onepiece['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [197]:
print(onepiece.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress',
       'smallCategoryName', 'colorNo', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'clothName', 'brand',
       'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')


In [198]:
onepiece = onepiece.drop('colorNo', axis=1)

In [199]:
# color도 모든 1의 값의 개수 == 행의 개수
num = 0
for i in range(7, 23):
    num += onepiece[onepiece.columns[i]].value_counts()[1]
    print(onepiece[onepiece.columns[i]].value_counts())
print(num)
print(onepiece.shape)

0    8744
1     704
Name: colorWhite, dtype: int64
0    8986
1     462
Name: colorGrey, dtype: int64
0    7389
1    2059
Name: colorBlack, dtype: int64
0    9204
1     244
Name: colorRed, dtype: int64
0    8985
1     463
Name: colorPink, dtype: int64
0    9329
1     119
Name: colorOrange, dtype: int64
0    8900
1     548
Name: colorIvory, dtype: int64
0    9106
1     342
Name: colorYellow, dtype: int64
0    8759
1     689
Name: colorGreen, dtype: int64
0    7966
1    1482
Name: colorBlue, dtype: int64
0    9212
1     236
Name: colorPurple, dtype: int64
0    9128
1     320
Name: colorBrown, dtype: int64
0    8570
1     878
Name: colorBeige, dtype: int64
0    9352
1      96
Name: colorJean, dtype: int64
0    9191
1     257
Name: colorPattern, dtype: int64
0    8899
1     549
Name: colorOthers, dtype: int64
9448
(9448, 42)


In [200]:
onepiece.insert(23, "colorName", True)

In [201]:
onepiece['colorName'] = onepiece.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
onepiece['colorName'] = onepiece.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

In [202]:
onepiece['colorName'].value_counts()

black      2059
blue       1482
beige       878
white       704
green       689
others      549
ivory       548
pink        463
grey        462
yellow      342
brown       320
pattern     257
red         244
purple      236
orange      119
jean         96
Name: colorName, dtype: int64

### Price 전처리

In [203]:
onepiece['clothPrice'] = onepiece['clothPrice'].apply(lambda x: x.replace(",", ""))
onepiece['clothPrice'] = onepiece['clothPrice'].apply(lambda x: x.replace("원", ""))
onepiece['clothPrice']

0        98000
1       127000
2       269000
3       125300
4       164000
         ...  
9443    111200
9444     63750
9445     89500
9446     85000
9447     71100
Name: clothPrice, Length: 9448, dtype: object

In [204]:
onepiece['clothPrice'] = onepiece['clothPrice'].astype('int')

In [205]:
onepiece['clothReviewNo'] = onepiece['clothReviewNo'].apply(lambda x: x.replace(",", ""))
onepiece['clothReviewNo'] = onepiece['clothReviewNo'].astype('int')
onepiece['clothReviewNo']

0       0
1       0
2       0
3       0
4       0
       ..
9443    0
9444    5
9445    0
9446    0
9447    2
Name: clothReviewNo, Length: 9448, dtype: int32

### Index를 NewGoodsNo로 변경

In [206]:
sub_onepiece = pd.read_json('./size_onepiece.json')
sub_onepiece

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,230853,none,23085304022,4022,[상큼],155.0,45.0,0.0,1.0,0,0,0,0,0,0,1,0,0,0,0,0
1,387218,그레이헤더/S,38721803234,3234,"[생각, 사이즈, 가도, 핏, 느낌, 참고]",161.0,43.0,0.0,1.0,0,0,0,0,0,0,0,1,0,0,0,0
2,435992,NONE,43599203203,3203,"[디자인, 맘, 드네, 멋, 입, 듯, 구김, 원단]",162.0,55.0,0.0,1.0,0,0,0,1,0,0,0,0,0,0,0,0
3,653344,none,65334404183,4183,"[스타일, 레깅스, 일리]",155.0,55.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,1,0
4,677397,FREE,67739703065,3065,"[입, 옷, 마음, 듭니]",163.0,55.0,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2042,2386153,M,23861533521,3521,"[원단, 생각, 여름, 듯, 무난, 무난, 디자인, 핏]",171.0,58.0,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0
2043,2396014,M,23960143328,3328,"[기장, 맘, 듭니]",173.0,53.0,0.0,1.0,0,0,1,0,0,0,0,0,0,0,0,0
2044,2396257,FREE,23962573722,3722,"[색상, 블랙, 팔이, 크게, 신경, 정도, 색상, 블랙, 팔이, 크게, 신경, 정도]",158.0,48.0,0.0,1.0,0,0,2,0,0,0,0,0,0,0,0,0
2045,2396259,FREE,23962593321,3321,"[카키색, 색상, 카키색, 색상]",158.0,48.0,0.0,1.0,0,0,2,0,0,0,0,0,0,0,0,0


In [207]:
onepiece = pd.merge(onepiece, sub_onepiece, how="left", left_on="clothId", right_on="goodsNo")
onepiece.shape

(9875, 64)

In [208]:
onepiece.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsNo', 'goodsSize', 'newGoodsNo',
       'index', 'reviewNoun', 'userHeight', 'userWeight', 'userSexMen',
       'userSexWomen', 'month1', 'month2', 'month3', 'month4', 'month5',
       'month6', 'month7', 'month8', 'month9', 'month10', 'month11',
       'month12'],
      dtype='object')

In [209]:
onepiece = onepiece.drop(["goodsNo", "index"], axis=1)
onepiece.shape

(9875, 62)

In [210]:
# 9875 : 반복문을 통해 변화된 skirt 행 개수
# 9448 : 원래의 skirt 행 개수
# 427 : 새로 생긴 skirt 행 개수
9875-9448

427

In [211]:
len(sub_onepiece)

2047

In [212]:
len(sub_onepiece.goodsNo.unique())

1620

In [213]:
from copy import deepcopy
b = deepcopy(onepiece)

In [214]:
b.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'newGoodsNo', 'reviewNoun',
       'userHeight', 'userWeight', 'userSexMen', 'userSexWomen', 'month1',
       'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8',
       'month9', 'month10', 'month11', 'month12'],
      dtype='object')

In [215]:
col_list_1 = ['newGoodsNo', 'userSexMen', 'userSexWomen']
col_list_2 = ['goodsSize', 'reviewNoun', 'userHeight', 'userWeight', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12']
check_list_1 = ['clothId',  'clothSexMen', 'clothSexWomen']
check_list_2 = ['Free', [], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(len(col_list_1)):
    b[col_list_1[i]] = b[col_list_1[i]].fillna(0)
    b[col_list_1[i]] = b.apply(lambda x: x[check_list_1[i]] if x[col_list_1[i]]==0 else x[col_list_1[i]], axis=1)
    if i==0:
        b[col_list_1[i]] = b[col_list_1[i]].astype('int64')
    
for i in range(len(col_list_2)):
    b[col_list_2[i]] = b[col_list_2[i]].fillna(0)
    b[col_list_2[i]] = b.apply(lambda x: check_list_2[i] if x[col_list_2[i]]==0 else x[col_list_2[i]], axis=1)
    if i == 0:
        b[col_list_2[i]] = b[col_list_2[i]].astype('str')
    elif i >= 3:
        b[col_list_2[i]] = b[col_list_2[i]].astype('int64')

In [216]:
onepiece = deepcopy(b)

In [217]:
# 이미지 null 값 확인
print(len(onepiece[onepiece.image.isnull()]))

0


In [218]:
# hashtags 전처리
onepiece.hashtags = onepiece.apply(lambda x: x.hashtags.replace("['", "").replace("']", "").replace("'", "").replace(" ", "").split(","), axis=1)

In [219]:
onepiece.userHeight = onepiece.userHeight.astype('float')
onepiece.userWeight = onepiece.userWeight.astype('float')
onepiece.userSexMen = onepiece.userSexMen.astype('float')
onepiece.userSexWomen = onepiece.userSexWomen.astype('float')

In [220]:
onepiece.reviewNoun.iloc[2]

[]

In [221]:
onepiece.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'newGoodsNo', 'reviewNoun',
       'userHeight', 'userWeight', 'userSexMen', 'userSexWomen', 'month1',
       'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8',
       'month9', 'month10', 'month11', 'month12'],
      dtype='object')

In [222]:
onepiece.columns = ['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'clothImg', 'clothPrice', 'date', 'clothHashtags', 'clothMale',
       'clothFemale', 'clothRate', 'clothReviewCnt', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'newClothId', 'reviewNoun', 'userHeight',
       'userWeight', 'userMale', 'userFemale', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12']

In [223]:
onepiece = onepiece[['newClothId', 'clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMinidress', 'smallCategoryMidi', 'smallCategoryMaxidress',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'clothImg', 'clothPrice', 'date', 'clothHashtags', 'clothMale',
       'clothFemale', 'clothRate', 'clothReviewCnt', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'reviewNoun', 'userHeight',
       'userWeight', 'userMale', 'userFemale', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12']]

In [224]:
onepiece.to_json('final_onepiece.json', orient='records', force_ascii=False)

In [225]:
onepiece.to_csv('final_onepiece.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')

# Skirt

In [226]:
skirt = pd.read_csv('./musinsa_clothes_skirt.csv')

In [227]:
skirt.columns

Index(['clothId', 'largeCategory', 'largeCategoryName', 'smallCategory',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [228]:
skirt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8030 entries, 0 to 8029
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   clothId            8030 non-null   int64  
 1   largeCategory      8030 non-null   int64  
 2   largeCategoryName  8030 non-null   object 
 3   smallCategory      8030 non-null   int64  
 4   smallCategoryName  8030 non-null   object 
 5   colorNo            8030 non-null   int64  
 6   clothName          8030 non-null   object 
 7   brand              8030 non-null   object 
 8   image              8030 non-null   object 
 9   clothPrice         8030 non-null   object 
 10  date               8030 non-null   int64  
 11  hashtags           8030 non-null   object 
 12  clothSexMen        8030 non-null   int64  
 13  clothSexWomen      8030 non-null   int64  
 14  clothSexCommon     8030 non-null   int64  
 15  clothRate          8030 non-null   float64
 16  clothReviewNo      8030 

In [229]:
skirt.describe()

,clothId,largeCategory,smallCategory,colorNo,date,clothSexMen,clothSexWomen,clothSexCommon,clothRate,fit,feeling,stretch,visibility,thickness,seasonSpring,seasonSummer,seasonFall,seasonWinter
count,8.030000e+03,8030.0,8030.00000,8030.000000,8.030000e+03,8030.0,8030.0,8030.0,8030.000000,8030.000000,8030.000000,8030.000000,8030.000000,8030.000000,8030.000000,8030.000000,8030.000000,8030.000000
mean,1.827303e+06,5.0,501.82142,14.868742,2.020636e+07,0.0,1.0,0.0,1.441557,0.921046,0.747572,0.918929,1.466874,0.951930,0.224782,0.158157,0.232628,0.144085
std,4.721685e+05,0.0,0.80569,15.687411,1.121140e+04,0.0,0.0,0.0,2.164618,1.388257,1.216880,1.549058,2.183245,1.477841,0.417465,0.364911,0.422533,0.351197
min,1.465310e+05,5.0,501.00000,1.000000,2.014112e+07,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.530896e+06,5.0,501.00000,2.000000,2.020073e+07,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.954120e+06,5.0,502.00000,7.000000,2.021052e+07,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.203965e+06,5.0,503.00000,24.000000,2.021103e+07,0.0,1.0,0.0,4.300000,2.000000,1.000000,1.000000,4.000000,3.000000,0.000000,0.000000,0.000000,0.000000
max,2.413365e+06,5.0,503.00000,71.000000,2.022031e+07,0.0,1.0,0.0,5.000000,5.000000,5.000000,5.000000,6.000000,5.000000,1.000000,1.000000,1.000000,1.000000


In [230]:
skirt = skirt.drop_duplicates(subset = "clothId")
skirt = skirt.reset_index(drop=True)

In [231]:
skirt['smallCategory'] = skirt['smallCategory'].astype('str')

In [232]:
skirt['largeCategory'].unique()

array([5], dtype=int64)

In [233]:
skirt['largeCategoryName'].unique()

array(['스커트'], dtype=object)

In [234]:
skirt['smallCategory'].unique()

array(['503', '501', '502'], dtype=object)

In [235]:
skirt['smallCategoryName'].unique()

array(['롱스커트', '미니스커트', '미디스커트'], dtype=object)

In [236]:
skirt['smallCategory'] = skirt['smallCategory'].astype('int')

### smallCategory 원핫인코딩 변환

In [237]:
skirt.insert(4, "smallCategoryLongskirt", skirt['smallCategory'], True)
skirt.insert(4, "smallCategoryMidi", skirt['smallCategory'], True)
skirt.insert(4, "smallCategoryMiniskirt", skirt['smallCategory'], True)

In [238]:
skirt['smallCategoryMiniskirt'] = skirt['smallCategoryMiniskirt'].apply(lambda x: 1 if x == 501 else 0)
skirt['smallCategoryMidi'] = skirt['smallCategoryMidi'].apply(lambda x: 1 if x == 502 else 0)
skirt['smallCategoryLongskirt'] = skirt['smallCategoryLongskirt'].apply(lambda x: 1 if x == 503 else 0)

In [239]:
skirt = skirt.drop('smallCategory', axis=1)

In [240]:
skirt.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothSexCommon', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')

In [241]:
# smallCategory 모든 1의 값의 개수 == 행의 개수

num = 0
for i in range(3, 6):
    num += skirt[skirt.columns[i]].value_counts()[1]
    print(skirt[skirt.columns[i]].value_counts())
print(num)
print(skirt.shape)

0    4573
1    3449
Name: smallCategoryMiniskirt, dtype: int64
0    5466
1    2556
Name: smallCategoryMidi, dtype: int64
0    6005
1    2017
Name: smallCategoryLongskirt, dtype: int64
8022
(8022, 28)


### clothSexCommon 변환

In [242]:
skirt['clothSexMen'] = skirt.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexMen'], axis=1)
skirt['clothSexWomen'] = skirt.apply(lambda x: 1 if x['clothSexCommon'] == 1 else x['clothSexWomen'], axis=1)

In [243]:
skirt = skirt.drop('clothSexCommon', axis=1)

In [244]:
skirt.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt',
       'smallCategoryName', 'colorNo', 'clothName', 'brand', 'image',
       'clothPrice', 'date', 'hashtags', 'clothSexMen', 'clothSexWomen',
       'clothRate', 'clothReviewNo', 'fit', 'feeling', 'stretch', 'visibility',
       'thickness', 'seasonSpring', 'seasonSummer', 'seasonFall',
       'seasonWinter'],
      dtype='object')

In [245]:
skirt['clothSexMen'] = skirt.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexMen'], axis=1)
skirt['clothSexWomen'] = skirt.apply(lambda x: 1 if (x['clothSexMen']==0 and x['clothSexWomen']==0) else x['clothSexWomen'], axis=1)

In [246]:
print(skirt['clothSexMen'].value_counts())
print(skirt['clothSexWomen'].value_counts())
print(skirt.shape)

0    8022
Name: clothSexMen, dtype: int64
1    8022
Name: clothSexWomen, dtype: int64
(8022, 27)


### color 원핫인코딩 변환

#### 스커트
- white (1)
- grey (13, 24, 3, 25)
- black (2)
- red (51, 11, 49)
- pink (47, 10, 45, 48, 55)
- orange (52, 54, 53, 46, 12)
- ivory (23)
- yellow (44, 9, 43, 14)
- green (33, 31, 32, 6, 34, 30, 35)
- blue (37, 38, 7, 36)
- purple (41, 39, 8, 40)
- brown (4, 27)
- beige (28, 26, 29, 5)
- jean (16, 57, 58, 59, 60)
- pattern (19, 20, 18, 61, 65, 62, 63, 64, 69, 66)
- others (71, 67, 70, 17, 21, 68, 22, 15)

In [247]:
skirt.insert(8, "colorOthers", skirt['colorNo'], True)
skirt.insert(8, "colorPattern", skirt['colorNo'], True)
skirt.insert(8, "colorJean", skirt['colorNo'], True)
skirt.insert(8, "colorBeige", skirt['colorNo'], True)
skirt.insert(8, "colorBrown", skirt['colorNo'], True)
skirt.insert(8, "colorPurple", skirt['colorNo'], True)
skirt.insert(8, "colorBlue", skirt['colorNo'], True)
skirt.insert(8, "colorGreen", skirt['colorNo'], True)
skirt.insert(8, "colorYellow", skirt['colorNo'], True)
skirt.insert(8, "colorIvory", skirt['colorNo'], True)
skirt.insert(8, "colorOrange", skirt['colorNo'], True)
skirt.insert(8, "colorPink", skirt['colorNo'], True)
skirt.insert(8, "colorRed", skirt['colorNo'], True)
skirt.insert(8, "colorBlack", skirt['colorNo'], True)
skirt.insert(8, "colorGrey", skirt['colorNo'], True)
skirt.insert(8, "colorWhite", skirt['colorNo'], True)

In [248]:
skirt['colorWhite'] = skirt['colorWhite'].apply(lambda x: 1 if x == 1 else 0)
skirt['colorGrey'] = skirt['colorGrey'].apply(lambda x: 1 if x in [13, 24, 3, 25] else 0)
skirt['colorBlack'] = skirt['colorBlack'].apply(lambda x: 1 if x == 2 else 0)
skirt['colorRed'] = skirt['colorRed'].apply(lambda x: 1 if x in [51, 11, 49] else 0)
skirt['colorPink'] = skirt['colorPink'].apply(lambda x: 1 if x in [47, 10, 45, 48, 55] else 0)
skirt['colorOrange'] = skirt['colorOrange'].apply(lambda x: 1 if x in [52, 54, 53, 46, 12] else 0)
skirt['colorIvory'] = skirt['colorIvory'].apply(lambda x: 1 if x == 23 else 0)
skirt['colorYellow'] = skirt['colorYellow'].apply(lambda x: 1 if x in [44, 9, 43, 14] else 0)
skirt['colorGreen'] = skirt['colorGreen'].apply(lambda x: 1 if x in [33, 31, 32, 6, 34, 30, 35] else 0)
skirt['colorBlue'] = skirt['colorBlue'].apply(lambda x: 1 if x in [37, 38, 7, 36] else 0)
skirt['colorPurple'] = skirt['colorPurple'].apply(lambda x: 1 if x in [41, 39, 8, 40] else 0)
skirt['colorBrown'] = skirt['colorBrown'].apply(lambda x: 1 if x in [4, 27] else 0)
skirt['colorBeige'] = skirt['colorBeige'].apply(lambda x: 1 if x in [28, 26, 29, 5] else 0)
skirt['colorJean'] = skirt['colorJean'].apply(lambda x: 1 if x in [16, 57, 58, 59, 60] else 0)
skirt['colorPattern'] = skirt['colorPattern'].apply(lambda x: 1 if x in [19, 20, 18, 61, 65, 62, 63, 64, 69, 66] else 0)
skirt['colorOthers'] = skirt['colorOthers'].apply(lambda x: 1 if x in [71, 67, 70, 17, 21, 68, 22, 15] else 0)

In [249]:
print(skirt.columns)

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt',
       'smallCategoryName', 'colorNo', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'clothName', 'brand',
       'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter'],
      dtype='object')


In [250]:
skirt = skirt.drop('colorNo', axis=1)

In [251]:
# color도 모든 1의 값의 개수 == 행의 개수
num = 0
for i in range(7, 23):
    num += skirt[skirt.columns[i]].value_counts()[1]
    print(skirt[skirt.columns[i]].value_counts())
print(num)
print(skirt.shape)

0    7545
1     477
Name: colorWhite, dtype: int64
0    7465
1     557
Name: colorGrey, dtype: int64
0    6157
1    1865
Name: colorBlack, dtype: int64
0    7835
1     187
Name: colorRed, dtype: int64
0    7666
1     356
Name: colorPink, dtype: int64
0    7960
1      62
Name: colorOrange, dtype: int64
0    7576
1     446
Name: colorIvory, dtype: int64
0    7874
1     148
Name: colorYellow, dtype: int64
0    7400
1     622
Name: colorGreen, dtype: int64
0    7040
1     982
Name: colorBlue, dtype: int64
0    7848
1     174
Name: colorPurple, dtype: int64
0    7642
1     380
Name: colorBrown, dtype: int64
0    7181
1     841
Name: colorBeige, dtype: int64
0    7700
1     322
Name: colorJean, dtype: int64
0    7685
1     337
Name: colorPattern, dtype: int64
0    7756
1     266
Name: colorOthers, dtype: int64
8022
(8022, 42)


In [252]:
skirt.insert(23, "colorName", True)

In [253]:
skirt['colorName'] = skirt.apply(lambda x: 'white' if x['colorWhite']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'grey' if x['colorGrey']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'black' if x['colorBlack']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'red' if x['colorRed']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'pink' if x['colorPink']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'orange' if x['colorOrange']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'ivory' if x['colorIvory']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'yellow' if x['colorYellow']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'green' if x['colorGreen']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'blue' if x['colorBlue']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'purple' if x['colorPurple']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'brown' if x['colorBrown']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'beige' if x['colorBeige']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'jean' if x['colorJean']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'pattern' if x['colorPattern']==1 else x['colorName'], axis=1)
skirt['colorName'] = skirt.apply(lambda x: 'others' if x['colorOthers']==1 else x['colorName'], axis=1)

In [254]:
skirt['colorName'].value_counts()

black      1865
blue        982
beige       841
green       622
grey        557
white       477
ivory       446
brown       380
pink        356
pattern     337
jean        322
others      266
red         187
purple      174
yellow      148
orange       62
Name: colorName, dtype: int64

### Price 전처리

In [255]:
skirt['clothPrice'] = skirt['clothPrice'].apply(lambda x: x.replace(",", ""))
skirt['clothPrice'] = skirt['clothPrice'].apply(lambda x: x.replace("원", ""))
skirt['clothPrice']

0       228000
1       156000
2       182000
3        88000
4       160000
         ...  
8017     99000
8018     89000
8019     32900
8020     51300
8021     58500
Name: clothPrice, Length: 8022, dtype: object

In [256]:
skirt['clothPrice'] = skirt['clothPrice'].astype('int')

In [257]:
skirt['clothReviewNo'] = skirt['clothReviewNo'].apply(lambda x: x.replace(",", ""))
skirt['clothReviewNo'] = skirt['clothReviewNo'].astype('int')
skirt['clothReviewNo']

0         0
1         0
2         0
3         0
4         0
       ... 
8017      1
8018      0
8019    148
8020     14
8021      0
Name: clothReviewNo, Length: 8022, dtype: int32

skirt.index = skirt['clothId']
skirt = skirt.drop('clothId', True)

In [258]:
type(skirt.hashtags.iloc[0])

str

### Index를 NewGoodsNo로 변경

In [259]:
sub_skirt = pd.read_json('./size_skirt.json')
sub_skirt

,goodsNo,goodsSize,newGoodsNo,index,reviewNoun,userHeight,userWeight,userSexMen,userSexWomen,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,387628,FREE/PINK,38762801116,1116,"[{'text': '리본', 'value': 1}, {'text': '포인트', '...",162.702128,51.744681,0.042553,0.957447,9,10,0,1,0,0,0,0,1,3,11,12
1,387631,FREE/BLUE,38763101604,1604,"[{'text': '생각', 'value': 1}, {'text': '색', 'va...",165.000000,50.000000,0.000000,1.000000,0,0,2,0,0,0,0,0,0,0,0,0
2,387632,FREE/NAVY,38763201044,1044,"[{'text': '마음', 'value': 2}, {'text': '듭니', 'v...",163.000000,50.000000,0.000000,1.000000,2,0,0,0,0,0,0,0,0,0,0,0
3,407626,FREE,4076260740,740,"[{'text': '배송', 'value': 1}, {'text': '색감', 'v...",165.000000,52.000000,0.000000,1.000000,0,1,0,0,0,0,0,0,0,0,0,0
4,463655,FREE,4636550465,465,"[{'text': '허리', 'value': 2}, {'text': '앞', 'va...",161.733333,53.933333,0.066667,0.933333,0,2,0,0,1,3,2,1,1,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649,2389731,FREE,23897312219,2219,"[{'text': '밴딩', 'value': 1}, {'text': '길이', 'v...",155.000000,50.000000,0.000000,1.000000,0,0,0,0,0,0,0,0,0,1,0,0
2650,2391576,S,23915762318,2318,"[{'text': '와우', 'value': 1}, {'text': '스타일', '...",159.625000,47.750000,0.125000,0.875000,4,3,1,0,0,0,0,0,0,0,0,0
2651,2391598,S,23915981607,1607,"[{'text': '핏', 'value': 1}, {'text': '것', 'val...",167.294118,65.647059,0.000000,1.000000,0,0,0,0,1,4,0,0,4,6,1,1
2652,2394978,M,2394978371,371,"[{'text': '전체', 'value': 1}, {'text': '색상', 'v...",164.000000,52.250000,0.000000,1.000000,0,0,0,0,0,0,0,0,0,0,3,1


In [267]:
skirt = pd.merge(skirt, sub_skirt, how="left", left_on="clothId", right_on="goodsNo")
skirt.shape

(8876, 64)

In [268]:
skirt.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsNo', 'goodsSize', 'newGoodsNo',
       'index', 'reviewNoun', 'userHeight', 'userWeight', 'userSexMen',
       'userSexWomen', 'month1', 'month2', 'month3', 'month4', 'month5',
       'month6', 'month7', 'month8', 'month9', 'month10', 'month11',
       'month12'],
      dtype='object')

In [269]:
skirt = skirt.drop(["goodsNo", "index"], axis=1)
skirt.shape

(8876, 62)

In [270]:
# 8876 : 반복문을 통해 변화된 skirt 행 개수
# 8022 : 원래의 skirt 행 개수
# 854 : 새로 생긴 skirt 행 개수
8876-8022

854

In [271]:
len(sub_skirt)

2654

In [272]:
len(sub_skirt.goodsNo.unique())

1800

In [273]:
from copy import deepcopy
b = deepcopy(skirt)

In [274]:
b.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'newGoodsNo', 'reviewNoun',
       'userHeight', 'userWeight', 'userSexMen', 'userSexWomen', 'month1',
       'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8',
       'month9', 'month10', 'month11', 'month12'],
      dtype='object')

In [275]:
col_list_1 = ['newGoodsNo', 'userSexMen', 'userSexWomen']
col_list_2 = ['goodsSize', 'reviewNoun', 'userHeight', 'userWeight', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12']
check_list_1 = ['clothId',  'clothSexMen', 'clothSexWomen']
check_list_2 = ['Free', [], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(len(col_list_1)):
    b[col_list_1[i]] = b[col_list_1[i]].fillna(0)
    b[col_list_1[i]] = b.apply(lambda x: x[check_list_1[i]] if x[col_list_1[i]]==0 else x[col_list_1[i]], axis=1)
    if i==0:
        b[col_list_1[i]] = b[col_list_1[i]].astype('int64')
    
for i in range(len(col_list_2)):
    b[col_list_2[i]] = b[col_list_2[i]].fillna(0)
    b[col_list_2[i]] = b.apply(lambda x: check_list_2[i] if x[col_list_2[i]]==0 else x[col_list_2[i]], axis=1)
    if i == 0:
        b[col_list_2[i]] = b[col_list_2[i]].astype('str')
    elif i >= 3:
        b[col_list_2[i]] = b[col_list_2[i]].astype('int64')

In [276]:
skirt = deepcopy(b)

In [277]:
# 이미지 null 값 확인
print(len(skirt[skirt.image.isnull()]))

0


In [278]:
# hashtags 전처리
skirt.hashtags = skirt.apply(lambda x: x.hashtags.replace("['", "").replace("']", "").replace("'", "").replace(" ", "").split(","), axis=1)

In [279]:
skirt.userHeight = skirt.userHeight.astype('float')
skirt.userWeight = skirt.userWeight.astype('float')
skirt.userSexMen = skirt.userSexMen.astype('float')
skirt.userSexWomen = skirt.userSexWomen.astype('float')

In [280]:
skirt.columns

Index(['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'image', 'clothPrice', 'date', 'hashtags', 'clothSexMen',
       'clothSexWomen', 'clothRate', 'clothReviewNo', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'newGoodsNo', 'reviewNoun',
       'userHeight', 'userWeight', 'userSexMen', 'userSexWomen', 'month1',
       'month2', 'month3', 'month4', 'month5', 'month6', 'month7', 'month8',
       'month9', 'month10', 'month11', 'month12'],
      dtype='object')

In [281]:
skirt.columns = ['clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'clothImg', 'clothPrice', 'date', 'clothHashtags', 'clothMale',
       'clothFemale', 'clothRate', 'clothReviewCnt', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'newClothId', 'reviewNoun', 'userHeight',
       'userWeight', 'userMale', 'userFemale', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12']

In [282]:
skirt = skirt[['newClothId', 'clothId', 'largeCategory', 'largeCategoryName',
       'smallCategoryMiniskirt', 'smallCategoryMidi', 'smallCategoryLongskirt',
       'smallCategoryName', 'colorWhite', 'colorGrey', 'colorBlack',
       'colorRed', 'colorPink', 'colorOrange', 'colorIvory', 'colorYellow',
       'colorGreen', 'colorBlue', 'colorPurple', 'colorBrown', 'colorBeige',
       'colorJean', 'colorPattern', 'colorOthers', 'colorName', 'clothName',
       'brand', 'clothImg', 'clothPrice', 'date', 'clothHashtags', 'clothMale',
       'clothFemale', 'clothRate', 'clothReviewCnt', 'fit', 'feeling',
       'stretch', 'visibility', 'thickness', 'seasonSpring', 'seasonSummer',
       'seasonFall', 'seasonWinter', 'goodsSize', 'reviewNoun', 'userHeight',
       'userWeight', 'userMale', 'userFemale', 'month1', 'month2',
       'month3', 'month4', 'month5', 'month6', 'month7', 'month8', 'month9',
       'month10', 'month11', 'month12']]

In [290]:
skirt.to_json('final_skirt.json', orient='records', force_ascii=False)

In [291]:
skirt.to_csv('final_skirt.csv',
                 sep=',',
                 float_format = '%.4f',
                 encoding = 'utf-8-sig')